Update: 1.1 is an update from "Train Retrieval and Adaptation Together". In the previous file, certain experiments' results are not accurate because a mistake I made in batch mode. This file redo everything.


This document is modified from "cars experiment 1.1". It is sufficiently different. 1.1 is about using neural network to do adaptation for regression task. This document, 1.2, is about using neural network to do both retrieval and adaptation for regression.

Regression model code based upon https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

Majority of the code adapted from [artificial data experiment with case removal.ipynb](https://colab.research.google.com/drive/13eN2qzSAB_wRqzzukBVjqcvt2GayzDCl?authuser=1#scrollTo=pRXm-C2PuCWe) 


Siamese network adapted from https://colab.research.google.com/drive/15wVnKzyQwWUD8xIpAstGx8N1ySnnmRvM?authuser=1

Also from https://towardsdatascience.com/siamese-networks-line-by-line-explanation-for-beginners-55b8be1d2fc6

Also refer https://www.kaggle.com/valerasarapas/keras-siamese-network-on-mnist
about "Plot the representation learned from the siamese network"

#Import

In [ ]:
import keras
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.utils import shuffle

from sklearn.neighbors import KNeighborsRegressor


In [ ]:
# Use some functions from tensorflow_docs
!pip install -q git+https://github.com/tensorflow/docs

     |████████████████████████████████| 1.0MB 5.9MB/s 


#Data

## Data (Cars)

We use already cleaned data from https://www.kaggle.com/bryanb/simple-and-quick-eda

IMPORTANT NOTE:: according to this post, there are a few outliers in the dataset, which might skew the learning. I am **not** tackling this now (e.g. removing them) and I want to see what happens.



Code adapted from https://www.kaggle.com/jshih7/car-price-prediction
to read data file and handle nominal attributes

also using a trick from https://stackoverflow.com/questions/18889588/create-dummies-from-column-with-multiple-values-in-pandas

In [ ]:
# filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/cars_cleaned.csv'
# filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/cars_cleaned_Above600kDeleted.csv'
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/cars_cleaned_Above600kDeleted_NoDuplicate.csv'

def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=0, usecols=(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,));

    y = pd.read_csv(filename,header=0, usecols=(16,));

    # X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    X = pd.concat([X, market_category_dummies], axis=1)
    X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=['Make', 'Model', 'Engine Fuel Type', 'Transmission Type', 'Driven_Wheels', 'Vehicle Size', 'Vehicle Style'] );
    
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y

Empty DataFrame
Columns: [Make, Model, Year, Engine Fuel Type, Engine HP, Engine Cylinders, Transmission Type, Driven_Wheels, Number of Doors, Market Category, Vehicle Size, Vehicle Style, highway MPG, city mpg, Popularity]
Index: []
Year                                0
Engine HP                           0
Engine Cylinders                    0
Number of Doors                     0
highway MPG                         0
                                   ..
Vehicle Style_Passenger Minivan     0
Vehicle Style_Passenger Van         0
Vehicle Style_Regular Cab Pickup    0
Vehicle Style_Sedan                 0
Vehicle Style_Wagon                 0
Length: 1009, dtype: int64


In [ ]:
X.head(5)

,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,Crossover,Diesel,Exotic,Factory Tuner,Flex Fuel,Hatchback,High-Performance,Hybrid,Luxury,No category,Performance,Make_Acura,Make_Alfa Romeo,Make_Aston Martin,Make_Audi,Make_BMW,Make_Bentley,Make_Buick,Make_Cadillac,Make_Chevrolet,Make_Chrysler,Make_Dodge,Make_FIAT,Make_Ferrari,Make_Ford,Make_GMC,Make_Genesis,Make_HUMMER,Make_Honda,Make_Hyundai,Make_Infiniti,Make_Kia,Make_Lamborghini,...,Model_iA,Model_iM,Model_iQ,Model_tC,Model_xA,Model_xB,Model_xD,Engine Fuel Type_0,Engine Fuel Type_1,Engine Fuel Type_2,Engine Fuel Type_3,Engine Fuel Type_4,Transmission Type_AUTOMATED_MANUAL,Transmission Type_AUTOMATIC,Transmission Type_DIRECT_DRIVE,Transmission Type_MANUAL,Transmission Type_UNKNOWN,Driven_Wheels_all wheel drive,Driven_Wheels_four wheel drive,Driven_Wheels_front wheel drive,Driven_Wheels_rear wheel drive,Vehicle Size_Compact,Vehicle Size_Large,Vehicle Size_Midsize,Vehicle Style_2dr Hatchback,Vehicle Style_2dr SUV,Vehicle Style_4dr Hatchback,Vehicle Style_4dr SUV,Vehicle Style_Cargo Minivan,Vehicle Style_Cargo Van,Vehicle Style_Convertible,Vehicle Style_Convertible SUV,Vehicle Style_Coupe,Vehicle Style_Crew Cab Pickup,Vehicle Style_Extended Cab Pickup,Vehicle Style_Passenger Minivan,Vehicle Style_Passenger Van,Vehicle Style_Regular Cab Pickup,Vehicle Style_Sedan,Vehicle Style_Wagon
0,2001,225.0,6,2,27,17,204,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,2002,225.0,6,2,27,17,204,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,2003,225.0,6,2,27,17,204,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,2003,260.0,6,2,27,17,204,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,2003,260.0,6,2,26,17,204,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
X.isna().sum()
nullseries =  X.isna().sum()
print(nullseries[nullseries > 0])

Series([], dtype: int64)


In [ ]:
X.shape

(6629, 1009)

In [ ]:
y.shape

(6629, 1)

In [ ]:
y.shape[0]

6629

In [ ]:
X[1:10]

,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,Crossover,Diesel,Exotic,Factory Tuner,Flex Fuel,Hatchback,High-Performance,Hybrid,Luxury,No category,Performance,Make_Acura,Make_Alfa Romeo,Make_Aston Martin,Make_Audi,Make_BMW,Make_Bentley,Make_Buick,Make_Cadillac,Make_Chevrolet,Make_Chrysler,Make_Dodge,Make_FIAT,Make_Ferrari,Make_Ford,Make_GMC,Make_Genesis,Make_HUMMER,Make_Honda,Make_Hyundai,Make_Infiniti,Make_Kia,Make_Lamborghini,...,Model_iA,Model_iM,Model_iQ,Model_tC,Model_xA,Model_xB,Model_xD,Engine Fuel Type_0,Engine Fuel Type_1,Engine Fuel Type_2,Engine Fuel Type_3,Engine Fuel Type_4,Transmission Type_AUTOMATED_MANUAL,Transmission Type_AUTOMATIC,Transmission Type_DIRECT_DRIVE,Transmission Type_MANUAL,Transmission Type_UNKNOWN,Driven_Wheels_all wheel drive,Driven_Wheels_four wheel drive,Driven_Wheels_front wheel drive,Driven_Wheels_rear wheel drive,Vehicle Size_Compact,Vehicle Size_Large,Vehicle Size_Midsize,Vehicle Style_2dr Hatchback,Vehicle Style_2dr SUV,Vehicle Style_4dr Hatchback,Vehicle Style_4dr SUV,Vehicle Style_Cargo Minivan,Vehicle Style_Cargo Van,Vehicle Style_Convertible,Vehicle Style_Convertible SUV,Vehicle Style_Coupe,Vehicle Style_Crew Cab Pickup,Vehicle Style_Extended Cab Pickup,Vehicle Style_Passenger Minivan,Vehicle Style_Passenger Van,Vehicle Style_Regular Cab Pickup,Vehicle Style_Sedan,Vehicle Style_Wagon
1,2002,225.0,6,2,27,17,204,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,2003,225.0,6,2,27,17,204,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,2003,260.0,6,2,27,17,204,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,2003,260.0,6,2,26,17,204,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,2015,150.0,4,4,35,24,204,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
6,2016,201.0,4,4,36,25,204,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7,2017,201.0,4,4,35,25,204,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8,2015,201.0,4,4,31,22,204,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
9,2014,111.0,4,4,38,39,204,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
y[1:10]

,MSRP
1,28030
2,28200
3,30550
4,30550
5,27050
6,27900
7,27990
8,29350
9,28900


##Data (Artificial)

In [ ]:
import more_itertools

def recursiveLists(dimension, values):
  if(dimension ==1):
    return list(range(values))
  else:
    totalList = []
    for eachVal in range(values):
      for eachSub in recursiveLists(dimension -1, values):
        tempList= [eachVal]
        tempList.append(eachSub)
        # print(tempList)
        totalList.append((tempList))
    return totalList
def flattenedRecurLists(dimension, values):
  aa = recursiveLists(dimension,values)
  bb = []
  for each in aa:
    bb.append(list(more_itertools.collapse(each)))
  return bb

In [ ]:
dimension = 4  #4 #2 #3
value_range = 6  #6 #30 #10

rows = flattenedRecurLists(dimension, value_range)

In [ ]:
#prepare X and y 
X = rows
y = [None]* len(rows)
for i in range(len(rows)):
  temp = 0
  for a in range(dimension):
    if rows[i][a] < value_range/2.0:
      temp += rows[i][a]
    else:
      temp += value_range - rows[i][a]
  y[i] = temp

X = np.array(X)
y = np.array(y)
y = y.reshape(y.shape[0],1)

In [ ]:
X.shape

(1296, 4)

In [ ]:
y.shape

(1296, 1)

In [ ]:
y.shape[0]

1296

In [ ]:
X[1:10]

array([[0, 0, 0, 1],
       [0, 0, 0, 2],
       [0, 0, 0, 3],
       [0, 0, 0, 4],
       [0, 0, 0, 5],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [0, 0, 1, 2],
       [0, 0, 1, 3]])

In [ ]:
y[1:10]

array([[1],
       [2],
       [3],
       [2],
       [1],
       [1],
       [2],
       [3],
       [4]])

In [ ]:
# import csv
# writer = None
# data = np.concatenate((X,y),axis = 1)
# with open('artificial.csv', 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile, delimiter=' ',
#                             quotechar='|', quoting=csv.QUOTE_MINIMAL)
#     writer.writerows(data)


refer to this while coding
https://keras.io/getting_started/intro_to_keras_for_engineers/
Code based on https://www.tensorflow.org/tutorials/keras/regression

Loading data and preprocessing

In [ ]:
X
X = X.astype(np.float)
X

array([[0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 2.],
       ...,
       [5., 5., 5., 3.],
       [5., 5., 5., 4.],
       [5., 5., 5., 5.]])

In [ ]:
y = y.astype(np.float)

In [ ]:
X.shape

(1296, 4)

In [ ]:
y.shape

(1296, 1)

In [ ]:
print(min(y), max(y))

[0.] [12.]


In [ ]:
X[1:10]

array([[0., 0., 0., 1.],
       [0., 0., 0., 2.],
       [0., 0., 0., 3.],
       [0., 0., 0., 4.],
       [0., 0., 0., 5.],
       [0., 0., 1., 0.],
       [0., 0., 1., 1.],
       [0., 0., 1., 2.],
       [0., 0., 1., 3.]])

##Data (Airfoil)

https://archive.ics.uci.edu/ml/datasets/airfoil+self-noise

In [ ]:
import csv
data = []
with open('/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/airfoil_self_noise.dat') as f:
  reader = csv.reader(f, delimiter="\t")
  for row in reader:
    data.append(row)
# print(data)
# print(len(data))
data = np.array(data)
print(data)
X = np.delete(data, 5, axis = 1)
y = data[:, 5]

X = X.astype(np.float)
y = y.astype(np.float)

[['800' '0' '0.3048' '71.3' '0.00266337' '126.201']
 ['1000' '0' '0.3048' '71.3' '0.00266337' '125.201']
 ['1250' '0' '0.3048' '71.3' '0.00266337' '125.951']
 ...
 ['4000' '15.6' '0.1016' '39.6' '0.0528487' '106.604']
 ['5000' '15.6' '0.1016' '39.6' '0.0528487' '106.224']
 ['6300' '15.6' '0.1016' '39.6' '0.0528487' '104.204']]


In [ ]:
X.shape

(1503, 5)

In [ ]:
y.shape

(1503,)

In [ ]:
y = y.reshape((-1,1))

In [ ]:
y.shape

(1503, 1)

In [ ]:
print(min(y), max(y))

[103.38] [140.987]


## Data (Student Performance)

https://archive.ics.uci.edu/ml/datasets/Student+Performance

In [ ]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/student-mat.csv'
# filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/student-por.csv'

def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=0, usecols=(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,
                                                16,17,18,19,20,21,22,23,24,25,26,27,28,29),sep=';');

    y = pd.read_csv(filename,header=0, usecols=(32,),sep=';');

    # X, y = shuffle(X, y)
    print(X.head(0))
    #binary attributes to boolean
    X['school'] = X['school'].map({'GP' : 0 , 'MS' : 1 })
    X['sex'] = X['sex'].map({'F' : 0 , 'M' : 1 })   
    X['address'] = X['address'].map({'U' : 0 , 'R' : 1 })
    X['famsize'] = X['famsize'].map({'LE3' : 0 , 'GT3' : 1 })
    X['Pstatus'] = X['Pstatus'].map({'T' : 0 , 'A' : 1 })
    variable_name = {'yes' : 0 , 'no' : 1 }
    # X['schoolsup,famsup,paid,activities,nursery,higher,internet,romantic'] = X['schoolsup,famsup,paid,activities,nursery,higher,internet,romantic'].map(variable_name)
    X['schoolsup'] = X['schoolsup'].map(variable_name)
    X['famsup'] = X['famsup'].map(variable_name)
    X['paid'] = X['paid'].map(variable_name)
    X['activities'] = X['activities'].map(variable_name)
    X['nursery'] = X['nursery'].map(variable_name)
    X['higher'] = X['higher'].map(variable_name)
    X['internet'] = X['internet'].map(variable_name)
    X['romantic'] = X['romantic'].map(variable_name)
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=['Mjob','Fjob','reason','guardian'] );
    
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y

Empty DataFrame
Columns: [school, sex, age, address, famsize, Pstatus, Medu, Fedu, Mjob, Fjob, reason, guardian, traveltime, studytime, failures, schoolsup, famsup, paid, activities, nursery, higher, internet, romantic, famrel, freetime, goout, Dalc, Walc, health, absences]
Index: []
school               0
sex                  0
age                  0
address              0
famsize              0
Pstatus              0
Medu                 0
Fedu                 0
traveltime           0
studytime            0
failures             0
schoolsup            0
famsup               0
paid                 0
activities           0
nursery              0
higher               0
internet             0
romantic             0
famrel               0
freetime             0
goout                0
Dalc                 0
Walc                 0
health               0
absences             0
Mjob_at_home         0
Mjob_health          0
Mjob_other           0
Mjob_services        0
Mjob_teacher         0
Fj

In [ ]:
X.head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other
0,0,0,18,0,1,1,4,4,2,2,0,0,1,1,1,0,0,1,1,4,3,4,1,1,3,6,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0
1,0,0,17,0,1,0,1,1,1,2,0,1,0,1,1,1,0,0,1,5,3,3,1,1,3,4,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0
2,0,0,15,0,0,0,1,1,1,2,3,0,1,0,1,0,0,0,1,4,3,2,2,3,3,10,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
3,0,0,15,0,1,0,4,2,1,3,0,1,0,0,0,0,0,0,0,3,2,2,1,1,5,2,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0
4,0,0,16,0,1,0,3,3,1,2,0,1,0,0,1,0,0,1,1,4,3,2,1,2,5,4,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0


In [ ]:
X.isna().sum()
nullseries =  X.isna().sum()
print(nullseries[nullseries > 0])

Series([], dtype: int64)


In [ ]:
X.shape

(395, 43)

In [ ]:
y.shape

(395, 1)

In [ ]:
y.shape[0]

395

In [ ]:
X[1:10]

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other
1,0,0,17,0,1,0,1,1,1,2,0,1,0,1,1,1,0,0,1,5,3,3,1,1,3,4,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0
2,0,0,15,0,0,0,1,1,1,2,3,0,1,0,1,0,0,0,1,4,3,2,2,3,3,10,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
3,0,0,15,0,1,0,4,2,1,3,0,1,0,0,0,0,0,0,0,3,2,2,1,1,5,2,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0
4,0,0,16,0,1,0,3,3,1,2,0,1,0,0,1,0,0,1,1,4,3,2,1,2,5,4,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0
5,0,1,16,0,0,0,4,3,1,2,0,1,0,0,0,0,0,0,1,5,4,2,1,2,5,10,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0
6,0,1,16,0,0,0,2,2,1,2,0,1,1,1,1,0,0,0,1,4,4,4,1,1,3,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0
7,0,0,17,0,1,1,4,4,2,2,0,0,0,1,1,0,0,1,1,4,1,4,1,1,1,6,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0
8,0,1,15,0,0,1,3,2,1,2,0,1,0,0,1,0,0,0,1,4,2,2,1,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0
9,0,1,15,0,1,0,3,4,1,2,0,1,0,0,0,0,0,0,1,5,5,1,1,1,5,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0


In [ ]:
y[1:10]

,G3
1,6
2,10
3,15
4,10
5,15
6,11
7,6
8,19
9,15


##Data (Face)

In [ ]:
import csv
data = []
# /content/drive/MyDrive/2020 Summer Assistantship CBR+Deep Learning/RBF+adaptation streamline/features.csv
with open('/content/drive/MyDrive/2020 Summer Assistantship CBR+Deep Learning/RBF+adaptation streamline/features.csv') as f:
  reader = csv.reader(f, delimiter=",")
  for row in reader:
    data.append(row)
X = np.array(data)

data = []
with open('/content/drive/MyDrive/2020 Summer Assistantship CBR+Deep Learning/RBF+adaptation streamline/target_classes.csv') as f:
  reader = csv.reader(f, delimiter=",")
  for row in reader:
    data.append(row)

y = np.array(data)

X = X.astype(np.float)
y = y.astype(np.float)

print(X[0])
print(y[0])

np.random.shuffle(X)
np.random.shuffle(y)

X = X[:,::8]
y = y[:,::8]

# limit = 6000
# X = X[:limit]
# y = y[:limit]

print(X[0])
print(y[0])

KeyboardInterrupt: ignored

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X[0]

In [ ]:
print(y)

##Data (Yacht Hydrodynamics)

https://archive.ics.uci.edu/ml/datasets/Yacht+Hydrodynamics#

In [ ]:
import csv
data = []
with open('/content/drive/MyDrive/2020 Summer Assistantship CBR+Deep Learning/RBF+adaptation streamline/yacht_hydrodynamics.data') as f:
  reader = csv.reader(f, delimiter=" ",skipinitialspace=True )
  i = 0
  for row in reader:
    # print(i)
    i = i +1
    data.append(np.asarray(row))

# print(len(data))
data = np.array(data)
# print(data)
X = np.delete(data, 6, axis = 1)
y = data[:, 6]

X = X.astype(np.float)
y = y.astype(np.float)

In [ ]:
X.shape

(308, 6)

In [ ]:
y.shape

(308,)

In [ ]:
y = y.reshape((-1,1))

In [ ]:
y.shape

(308, 1)

## Data (Energy Efficiency- Heating)

https://archive.ics.uci.edu/ml/datasets/Energy+efficiency#

In [ ]:
filename = '/content/drive/MyDrive/2020 Summer Assistantship CBR+Deep Learning/RBF+adaptation streamline/ENB2012_data.csv'

def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=0, usecols=(0,1,2,3,4,5,6,7,));
    # Change this to 9, to use a different class label, (cooling)
    y = pd.read_csv(filename,header=0, usecols=(8,));

    # X, y = shuffle(X, y)
    print(X.head(0))
    #normal one hot encoding
    
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y

Empty DataFrame
Columns: [X1, X2, X3, X4, X5, X6, X7, X8]
Index: []
X1    0
X2    0
X3    0
X4    0
X5    0
X6    0
X7    0
X8    0
dtype: int64


In [ ]:
X.head(5)

,X1,X2,X3,X4,X5,X6,X7,X8
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0


In [ ]:
X.isna().sum()
nullseries =  X.isna().sum()
print(nullseries[nullseries > 0])

Series([], dtype: int64)


In [ ]:
X.shape

(768, 8)

In [ ]:
y.shape

(768, 1)

In [ ]:
y.shape[0]

768

In [ ]:
X[1:10]

,X1,X2,X3,X4,X5,X6,X7,X8
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0
5,0.90,563.5,318.5,122.50,7.0,3,0.0,0
6,0.90,563.5,318.5,122.50,7.0,4,0.0,0
7,0.90,563.5,318.5,122.50,7.0,5,0.0,0
8,0.86,588.0,294.0,147.00,7.0,2,0.0,0
9,0.86,588.0,294.0,147.00,7.0,3,0.0,0


In [ ]:
y[1:10]

,Y1
1,15.55
2,15.55
3,15.55
4,20.84
5,21.46
6,20.71
7,19.68
8,19.50
9,19.95


#Preparation and global parameters

In [ ]:
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0
beta = 1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = -1

In [ ]:
X.shape

(1503, 5)

In [ ]:
# split_random = 42
split_random = np.random.randint(100)
np.random.seed(split_random)
from tensorflow.keras import regularizers

In [ ]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
# 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. Note that the progress bar is not particularly useful when logged to a file, so verbose=2 is recommended when not running interactively (eg, in a production environment).
verbose_param = 0
global_patience = 10
# right now there is an issue that we might overfit models because early_stop won't happen unless we train models more per epoch
# we need adapt_per_train_epoch > global_patience, 
# but for now, let's roll with this because the model may converge very fast.
train_valid_ratio = 0.9
freezing_sim_when_training_adapt = False

Debug setting

In [ ]:

retrieval_debug = False

#Standardize and Split Data

maybe we can preprocess numerical values: normalization and standardization

Based on https://machinelearningmastery.com/how-to-improve-neural-network-stability-and-modeling-performance-with-data-scaling/

According to: https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/
They used StandardScaler.

IMPORTANT NOTE: Scaling is done on whole dataset instead of on train only. This might be a data leakage issue. However I intentionally do it for now because there is several outlier. If the outliers unluckily all fall in train or testing set, it is going to cause a problem, so I scale them together. 


In [ ]:
# demonstrate data standardization with sklearn
from sklearn.preprocessing import StandardScaler
# create scaler
Xscaler = StandardScaler()
# fit scaler on data
Xscaler.fit(X)
# apply transform
X = Xscaler.transform(X)
# inverse transform
inverse = Xscaler.inverse_transform(X)

In [ ]:
y.mean()

124.83594278110448

In [ ]:
y.std()

6.896361277488791

In [ ]:
yscaler = StandardScaler()

yscaler.fit(y)
y = yscaler.transform(y)

In [ ]:
y.mean()

-2.9688678513462602e-15

In [ ]:
y.std()

0.9999999999999999

Split to train and test. random_state involved, so it might produce different results if using default value.

Not using cross validation here yet for simplicity. This is a todo for the future


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, random_state= split_random)



Main Code

In [ ]:
x_dimension = X.shape[1]

In [ ]:
X_train.shape

(1352, 5)

# Defining the models


In [ ]:
# modified from https://keras.io/examples/mnist_siamese/

from __future__ import absolute_import
from __future__ import print_function
import numpy as np

import random
from keras.datasets import fashion_mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda, Concatenate, Subtract
from keras.optimizers import RMSprop
from keras import backend as K

#Global Parameters here

# max_distance = max(y) - min(y)
#this is made to be 2 because y labels are standardized
max_distance = 2
input_shape = X_train.shape[1:]
feature_dim = 128
sim_loss_option = 'mimic_adapt' 
# sim_loss_option = 'cross_entropy'
contrastive_loss_margin = 0.2 #tentative


def euclidean_distance(vects):
  x, y = vects
  #assuming all attributes are between 0-1. The activation function for the attribtues are sigmoid.
  sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
  #slightly modified, to keep it between 0 and 1.
  #MAYBE: this distance / dimension * range of CC
  distance = K.sqrt(K.maximum(sum_square, K.epsilon()))
  #range: 0-
  return distance

def element_wise_distance(vects):
  x, y = vects
  return K.abs(x-y)

def eucl_dist_output_shape(shapes):
  shape1, shape2 = shapes
  return (shape1[0], 1)

#If using this version, the output of siamese is "how different two inputs are"
#y_true is 1 if the inputs are similar. 0 if very different
def contrastive_loss(y_true, y_pred):
  #limit the range to 0-1
  # y_pred = y_pred / sqrt(feature_dim)
  '''Contrastive loss from Hadsell-et-al.'06
  http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
  '''
  margin = contrastive_loss_margin
  square_pred = K.square(y_pred)
  margin_square = K.square(K.maximum(margin - y_pred, 0))
  return K.mean(float(y_true) * square_pred + float(1 - y_true) * margin_square)

#If using this loss, the output of siamese is mimicking the adaptation
def mse_loss(y_true, y_pred):
  #limit the range to 0-1
  # y_pred = y_pred / sqrt(feature_dim)
  return K.square(y_true - y_pred)


def mse_error(y_true, y_pred):
  return np.mean(np.square(y_true - y_pred)) #default np.mean compute mean of the flattened array


##Hard model

In [ ]:
if(hard_or_easy == 'hard'):
  feature_dim = 64
  def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
    x = Dropout(0.1)(x)
    #using sigmoid instead of relu to limit the range of extracted features
    x = Dense(feature_dim, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)#trying relu here. it was sigmoid
    return Model(input, x)


  #TODO IMPORTANT, We will need to test out two different metric and which one is better. To assess which is better, we can rely on MSE loss of the final result (retrieval + adaptation)

  base_network = create_base_network(input_shape)

  # similarity network definition
  input_a = Input(shape=input_shape)
  input_b = Input(shape=input_shape)
  # because we re-use the same instance `base_network`,
  # the weights of the network
  # will be shared across the two branches
  processed_a = base_network(input_a)
  processed_b = base_network(input_b)

  # distance = Lambda(euclidean_distance,
  #                   output_shape=eucl_dist_output_shape)([processed_a, processed_b])

  #IMPORTANT, I am using subtract + sigmoid layer to do similarity. This is inspired by https://towardsdatascience.com/siamese-networks-line-by-line-explanation-for-beginners-55b8be1d2fc6
  #By doing this, I avoid defining the metric. Also I avoid the problem of setting threshold for when I am using contrastive loss.
  #By doing this, I also don't need to limit the range of y_pred in the loss functions
  # subtracted = keras.layers.Subtract()([processed_a, processed_b])
  # subtracted = keras.layers.Absolute()(subtracted)
  subtracted = Lambda(element_wise_distance,
                    output_shape=eucl_dist_output_shape)([processed_a, processed_b])

  distance = Dense(1, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(subtracted)#switching to relu. sigmoid won't work!!! I thought it is going to train a biass somehow, but apparently not.
  #sim model output range 0-1
  sim_model = Model([input_a, input_b], distance)
  #compile
  rms = RMSprop()
  if sim_loss_option == 'mimic_adapt':
    sim_model.compile(loss=mse_loss, optimizer=rms)
  elif sim_loss_option == 'cross_entropy':
    sim_model.compile(loss=contrastive_loss, optimizer = rms)

  input_a_adp = Input(shape=input_shape)
  input_b_adp = Input(shape=input_shape)
  base_network_adp = create_base_network(input_shape)
  processed_a_adp = base_network_adp(input_a_adp)
  processed_b_adp = base_network_adp(input_b_adp)
  #adaptation network definition
  concat = Concatenate()([processed_a_adp, processed_b_adp])
  x = Dense(128, input_dim=(feature_dim*2), kernel_initializer='normal', activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(concat)
  #the adapt network is different from cars experiment 1.1, here we use dropout and less layers.
  x = Dense(128, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
  x = Dropout(0.1)(x)
  x = Dense(64, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
  x = Dropout(0.1)(x)
  #IMPORTANT:: I used linear activation function here, not sure if this is right.
  out = Dense(1, kernel_initializer='normal',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
  adapt_model = Model([input_a_adp,input_b_adp], out)

  # Compile model
  adapt_model.compile(loss='mean_squared_error', optimizer='adam')
  #adapt_model output range is about -1 ~ 1 (not rigid because last layer is linear activation.)

##Easy Model 

In [ ]:
if(hard_or_easy == 'easy'):
  feature_dim = 16
  def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(32, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
    x = Dropout(0.1)(x)
    #using sigmoid instead of relu to limit the range of extracted features
    x = Dense(feature_dim, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)#trying relu here. it was sigmoid
    return Model(input, x)


  #TODO IMPORTANT, We will need to test out two different metric and which one is better. To assess which is better, we can rely on MSE loss of the final result (retrieval + adaptation)

  base_network = create_base_network(input_shape)

  # similarity network definition
  input_a = Input(shape=input_shape)
  input_b = Input(shape=input_shape)
  # because we re-use the same instance `base_network`,
  # the weights of the network
  # will be shared across the two branches
  processed_a = base_network(input_a)
  processed_b = base_network(input_b)

  # distance = Lambda(euclidean_distance,
  #                   output_shape=eucl_dist_output_shape)([processed_a, processed_b])

  #IMPORTANT, I am using subtract + sigmoid layer to do similarity. This is inspired by https://towardsdatascience.com/siamese-networks-line-by-line-explanation-for-beginners-55b8be1d2fc6

  subtracted = Lambda(element_wise_distance,
                    output_shape=eucl_dist_output_shape)([processed_a, processed_b])

  distance = Dense(1, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(subtracted)#switching to relu. sigmoid won't work!!! I thought it is going to train a biass somehow, but apparently not.
  #sim model output range 0-1
  sim_model = Model([input_a, input_b], distance)
  #compile
  rms = RMSprop()
  if sim_loss_option == 'mimic_adapt':
    sim_model.compile(loss=mse_loss, optimizer=rms)
  elif sim_loss_option == 'cross_entropy':
    sim_model.compile(loss=contrastive_loss, optimizer = rms)

  input_a_adp = Input(shape=input_shape)
  input_b_adp = Input(shape=input_shape)
  base_network_adp = create_base_network(input_shape)
  processed_a_adp = base_network_adp(input_a_adp)
  processed_b_adp = base_network_adp(input_b_adp)
  #adaptation network definition
  concat = Concatenate()([processed_a_adp, processed_b_adp])
  x = Dense(32, input_dim=(feature_dim*2), kernel_initializer='normal', activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(concat)
  #the adapt network is different from cars experiment 1.1, here we use dropout and less layers.
  x = Dense(32, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
  x = Dropout(0.1)(x)
  x = Dense(16, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
  x = Dropout(0.1)(x)
  #IMPORTANT:: I used linear activation function here, not sure if this is right.
  out = Dense(1, kernel_initializer='normal',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
  adapt_model = Model([input_a_adp,input_b_adp], out)

  # Compile model
  adapt_model.compile(loss='mean_squared_error', optimizer='adam')
  #adapt_model output range is about -1 ~ 1 (not rigid because last layer is linear activation.)

#Train the models

Now we have sim_model and adapt_model, let's train them.

##X pair assemble

In [ ]:
def assemble_pairs_from_retrieval(retrieved_X, query_X):
  retrieve_size = len(retrieved_X)
  retrieved_X_pairs = []
  retrieved_X_pairs.append(np.zeros((retrieve_size, input_shape[0])))
  retrieved_X_pairs.append(np.zeros((retrieve_size, input_shape[0])))
  for i in range(0, retrieve_size):
    #IMPORTANT, left source, right target, this is different from 1.1
    retrieved_X_pairs[0][i] = retrieved_X[i]
    retrieved_X_pairs[1][i] = query_X
  retrieved_X_pairs = np.array(retrieved_X_pairs)
  return retrieved_X_pairs

## Retrieval process

In [ ]:
knn = KNeighborsRegressor(n_neighbors=retrieve_size)
knn.fit(X_train, y_train)

def retrieve(query_X, query_y, retrieve_from_X, retrieve_from_y):
  retrieved_X, retrieved_y = None, None
  if(disable_retrieval):
    
    # y_test_pred = knn.predict(X_test)  
    # knn_loss = mean_squared_error(y_test, y_test_pred, squared=TrueF)
    query_X_temp = query_X.reshape(1, -1)
    X_test_retrieved_index = knn.kneighbors(query_X_temp, n_neighbors=retrieve_size)[1]
    #we only select the best neighbor here!!!
    X_test_retrieved_index = X_test_retrieved_index[:retrieve_size][0] #0]
    # print(X_test_retrieved_index)
    retrieved_X = X_train[X_test_retrieved_index]
    retrieved_y = y_train[X_test_retrieved_index]
    # print(retrieved_X)
  else:

    #we want the retrieval process to sort retrieve_from_X
    #argsort the scores to get the indices https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
    retrieve_from_X_pairs = assemble_pairs_from_retrieval(retrieve_from_X, query_X)
    sim_scores = sim_model.predict([retrieve_from_X_pairs[0],retrieve_from_X_pairs[1]])

    sorted_score_index = np.argsort(sim_scores,axis=None) #axis=None will flatten the array
    #sort the retrieve pool by sim_score
    retrieve_from_X = retrieve_from_X[sorted_score_index]
    retrieve_from_y = retrieve_from_y[sorted_score_index]

    #retrieve good ones according to sim model.
    retrieved_X = retrieve_from_X[:retrieve_size]
    retrieved_y = retrieve_from_y[:retrieve_size]

    if(retrieval_debug):
      # print("sorted_score_index", sorted_score_index)
      # print("sim_scores", sim_scores[sorted_score_index])
      print("retrieved_X[0]", retrieved_X[0])
      print("retrieved_y[0]", retrieved_y[0])

  retrieved_X_pairs = assemble_pairs_from_retrieval(retrieved_X, query_X)
  query_y_1 = np.array([query_y,] * len(retrieved_y))
  #IMPORTANT, left source, right target, this is different from 1.1
  y_adapt_diff = np.subtract(query_y_1, retrieved_y)

  #also retrieve random bad ones.
  if(bad_retrieve_size != 0):
    #NOTE: the bad ones can be retrieved randomly. If we use the worst ones, then the pairs are too easy.
    #If we use random ones, the pairs will be harder and thus provide more info.
    bad_pairs_indices = np.random.choice(np.arange(len(retrieve_from_X)-retrieve_size), bad_retrieve_size, replace = False)
    retrieved_X_2 = retrieve_from_X[retrieve_size:][bad_pairs_indices]
    retrieved_y_2 = retrieve_from_y[retrieve_size:][bad_pairs_indices]
    retrieved_X_pairs_2 = assemble_pairs_from_retrieval(retrieved_X_2, query_X)
    query_y_2 = np.array([query_y,] * len(retrieved_y_2))
    y_adapt_diff_2 = np.subtract(query_y_2, retrieved_y_2)

    retrieved_X_pairs = np.concatenate((retrieved_X_pairs, retrieved_X_pairs_2),axis=1)
    retrieved_y = np.vstack((retrieved_y, retrieved_y_2))
    y_adapt_diff = np.vstack((y_adapt_diff, y_adapt_diff_2))
    query_y = np.vstack((query_y_1,query_y_2))
  else:
    query_y = query_y_1
  return retrieved_X_pairs, retrieved_y, y_adapt_diff, query_y
#note: y_adapt_diff is the difference between their real y solutions

##Split of train/test/validation

In [ ]:
#split by index
def split_train_valid_for_pairs(X_pairs, y_pairs):
  pair_size = X_pairs.shape[1]
  permutated_indices = np.random.permutation(np.arange(pair_size))
  cutoff = int(pair_size*train_valid_ratio)
  X_pair_train = [X_pairs[0][permutated_indices[:cutoff]],X_pairs[1][permutated_indices[:cutoff]]]
  y_pair_train = y_pairs[permutated_indices[:cutoff]]
  X_pair_valid = [X_pairs[0][permutated_indices[cutoff:]],X_pairs[1][permutated_indices[cutoff:]]]
  y_pair_valid = y_pairs[permutated_indices[cutoff:]]
  return X_pair_train, y_pair_train, X_pair_valid, y_pair_valid

def prepare_pairs(X_pairs, y_pairs):
  pair_size = X_pairs.shape[1]
  permutated_indices = (np.arange(pair_size))
  X_pair_train = [X_pairs[0][permutated_indices],X_pairs[1][permutated_indices]]
  y_pair_train = y_pairs[permutated_indices]
  return X_pair_train, y_pair_train

In [ ]:
# def reset_base_network_trainable(flag):
#   base_network.trainable = flag
#   #recompile models
#   if sim_loss_option == 'mimic_adapt':
#     sim_model.compile(loss=mse_loss, optimizer=rms)
#   elif sim_loss_option == 'cross_entropy':
#     sim_model.compile(loss=contrastive_loss, optimizer = rms)
#   adapt_model.compile(loss='mean_squared_error', optimizer='adam')


##Sim_model distance calculator

In [ ]:
def sim_model_label(sim_loss_option, query_y_temp, y_pred):
  temp_y_pair_real = None
  if sim_loss_option == 'mimic_adapt':
    #using absolute distance here, because sim_model last layer is an euclidean distance
    temp_y_pair_real = np.absolute(np.subtract(query_y_temp, y_pred))
  elif sim_loss_option == 'cross_entropy':
    #y_true is 1 if the inputs are similar. 0 if very different
    abs_distances = np.absolute(np.subtract(query_y_temp, y_pred))
    temp_y_pair_real = [1 if abs_distances[i] < contrastive_loss_margin else 0 for i in range(abs_distances.shape[0])]

  temp_y_pair_real = np.array(temp_y_pair_real)
  return temp_y_pair_real

In [ ]:
#NOT USED
# def sim_model_loss(sim_loss_option, query_y_temp, y_pred, y_retrieved):
#   y_pair_real = None
#   if sim_loss_option == 'mimic_adapt':
#     #using absolute distance here, because sim_model last layer is an euclidean distance
#     y_pair_real = np.absolute(np.subtract(query_y_temp, y_pred))
#   elif sim_loss_option == 'cross_entropy':
#     #y_true is 1 if the inputs are similar. 0 if very different
#     abs_distances = np.absolute(np.subtract(query_y_temp, y_pred))
#     y_pair_real = [1 if abs_distances[i] < contrastive_loss_margin else 0 for i in range(abs_distances.shape[0])]

#   y_pair_real = np.array(y_pair_real)
#   y_pair_real = np.concatenate(())
#   return y_pair_real

##Checkpoint and Loss function

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

sim_model_file = 'sim_model_weights'
sim_checkpointer = ModelCheckpoint(
    filepath= sim_model_file
    , save_weights_only=True
    , monitor = "val_loss"
    , verbose=1
    , save_best_only=True
    , mode = 'auto'
)

adapt_model_file = 'adapt_model_weights'
adapt_checkpointer = ModelCheckpoint(
    filepath= adapt_model_file
    , save_weights_only=True
    , monitor = "val_loss"
    , verbose=1
    , save_best_only=True
    , mode = 'auto'
)


Cost function is inspired by this:
https://stackoverflow.com/questions/59217549/how-to-use-multiple-variables-as-loss-for-a-model-with-one-output-in-keras

In [ ]:
# def loss(y_true, y_pred): #where true is shape (batch, 2) and pred is shape (batch, 1)
#     return K.mean(K.square(y_true - y_pred), axis=-1)

# def loss(y_true, y_pred): #where true is shape (batch, 2) and pred is shape (batch,)
#     return K.square(y_pred - y_true[:,0]) + K.square(y_pred - y_true[:,1])

# def loss(y_true, y_pred): #where true is shape (batch, 2) and pred is shape (batch,)
#     y_pred = K.reshape(y_pred, (-1,1))
#     return K.mean(K.square(y_true - y_pred), axis=-1)

#pre-adaptation distance = |retrieved y - query y|
#post-adaptation distance = |adapted y - query y|

#DOING, special loss function, alpha *retrieval loss + beta * adaptation loss
#y_pred shape (batch,)
#y_pred = distance prediction of sim_model
#y_true shape (batch, 2)
#y_true = pre-adaptation distance  + post-adaptation distance


# def sim_loss(y_true, y_pred):
#   return (alpha* K.square(y_pred - y_true) )

def sim_loss(y_true, y_pred):
  return (alpha* K.square(y_pred -tf.reshape(y_true[:,0],[-1,1])) + beta* K.square(y_pred - tf.reshape(y_true[:,1],[-1,1])))

def sim_loss_with_margin(y_true, y_pred):
  return (alpha* K.square(K.maximum(0.0, K.abs(y_pred - tf.reshape(y_true[:,0],[-1,1]))-alpha_margin ))+ beta* K.square(y_pred - tf.reshape(y_true[:,1],[-1,1])))

if(using_margin):
  sim_model.compile(loss= sim_loss_with_margin, optimizer=rms)
else:
  sim_model.compile(loss=sim_loss, optimizer=rms)

def reset_base_network_trainable(flag):
  base_network.trainable = flag
  #recompile models
  # sim_model.compile(loss=sim_loss, optimizer=rms)
  if(using_margin):
    sim_model.compile(loss= sim_loss_with_margin, optimizer=rms)
  else:
    sim_model.compile(loss=sim_loss, optimizer=rms)
  adapt_model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# y_pre_adapt

In [ ]:
# y_post_adapt

In [ ]:
# y_pair_real = np.concatenate((y_pre_adapt, y_post_adapt), axis = 1)

In [ ]:
# y_pair_real

In [ ]:
# y_pair_real[:,0]

In [ ]:
sim_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 16)           1776        input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 16)           0           model[0][0]                

In [ ]:
# y_pair_real[:,0].reshape(-1,1).shape

In [ ]:
# import tensorflow as tf

# tf.keras.backend.get_value(K.square(y_pre_adapt - y_pair_real[:,0].reshape(-1,1))) 



###Training process case by case

In [ ]:
# if(batch_mode):
#   print("In batch mode")

# query_indices = np.random.permutation(np.arange(X_train.shape[0]))
# if(num_of_training_queries != -1):
#   query_indices = query_indices[:num_of_training_queries]
# for query_index in query_indices:
#   #for every query case
#   query_X = X_train[query_index]
#   query_y = y_train[query_index]

#   indices = np.arange(len(X_train))

#   #TODO:: to speed up, for efficiency, maybe search/retrieve fewer cases
#   retrieve_from_X = X_train[indices!=query_index,:]
#   retrieve_from_y = y_train[indices!=query_index,:]

#   for i in range(back_forth):
#     #search from stored cases and compute similarity
#     retrieved_X_pairs, retrieved_y, y_adapt_diff, query_y_temp = retrieve(query_X, query_y, retrieve_from_X, retrieve_from_y)
#     if freezing_sim_when_training_adapt:
#       reset_base_network_trainable(False)
#     #train adapt model
#     early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=global_patience, verbose=verbose_param)
#     X_adapt_train, y_adapt_train, X_adapt_valid, y_adapt_valid = split_train_valid_for_pairs(retrieved_X_pairs,y_adapt_diff)
#     early_history = adapt_model.fit(X_adapt_train, y_adapt_train, 
#                         epochs=adapt_per_train_epoch, validation_data=(X_adapt_valid, y_adapt_valid), verbose=verbose_param, 
#                         callbacks=[early_stop, tfdocs.modeling.EpochDots()])
#     print(early_history)
#     # print("adapt model last loss: ",early_history.history['val_loss'][-1])

#     # get CC to train sim model.
#     y_adapt_diff_pred = adapt_model.predict([retrieved_X_pairs[0],retrieved_X_pairs[1]])
#     y_pred = retrieved_y + y_adapt_diff_pred

#     #true CC to train sim model
#     y_pair_real = sim_model_label(sim_loss_option, query_y_temp, y_pred)
#     #split
#     X_sim_train, y_sim_train, X_sim_valid, y_sim_valid = split_train_valid_for_pairs(retrieved_X_pairs,y_pair_real)
#     if freezing_sim_when_training_adapt:
#       reset_base_network_trainable(True)

#     #train sim model
#     early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=global_patience, verbose=verbose_param)
#     early_history = sim_model.fit(X_sim_train, y_sim_train, 
#                         epochs=sim_per_train_epoch, validation_data=(X_sim_valid, y_sim_valid), verbose=verbose_param, 
#                         callbacks=[early_stop, tfdocs.modeling.EpochDots()])
    


Note: the loss and val_loss often jumps up and down. Maybe longer training epochs for each model are better?

After training, test the whole process

###Training Process in Batch

This should speed up the process. If we do multiple retrievals per query, and do multiple queries per batch. Then batch optimization will help us to run faster.

Note that the back-n-forth process cannot be saved, and it will be slow. But batch optimization is built-in in the backpropagation process.

In [ ]:
# #IMPORTANT ASSUMPTION:
# #This implementation only works with the following parameters 
# retrieve_size = 1
# bad_retrieve_size = 0

# if(not batch_mode):
#   print("not in batch mode")

# query_indices = np.random.permutation(np.arange(X_train.shape[0]))
# if(num_of_training_queries != -1):
#   query_indices = query_indices[:num_of_training_queries]

# query_counter = 0
# back_forth_counter = 0
# for query_counter in range(0,len(query_indices),batch_size):
#   for i in range(back_forth):
#     retrieved_X_pairs, retrieved_y, y_adapt_diff, query_y_temp = np.zeros((2,0,1009)),np.zeros((0,1)),np.zeros((0,1)),np.zeros((0,1))
#     for j in range(batch_size):
#       query_index = query_indices[query_counter + j]
#       #for every query case
#       query_X = X_train[query_index]
#       query_y = y_train[query_index]

#       indices = np.arange(len(X_train))
#       #TODO:: to speed up, for efficiency, maybe search/retrieve fewer cases
#       retrieve_from_X = X_train[indices!=query_index,:]
#       retrieve_from_y = y_train[indices!=query_index,:]
#       #prepare batch
#       temp_retrieved_X_pairs, temp_retrieved_y, temp_y_adapt_diff, temp_query_y_temp = retrieve(query_X, query_y, retrieve_from_X, retrieve_from_y)
      
#       retrieved_X_pairs = np.concatenate((retrieved_X_pairs, temp_retrieved_X_pairs),axis=1)#np.vstack((retrieved_X_pairs, temp_retrieved_X_pairs))
#       print(retrieved_y.shape, temp_retrieved_y.shape)
#       retrieved_y = np.vstack((retrieved_y, temp_retrieved_y))
#       y_adapt_diff = np.vstack((y_adapt_diff, temp_y_adapt_diff))
#       query_y_temp = np.vstack((query_y_temp, temp_query_y_temp))

#     #actual training when a batch is ready
#     #train adapt_model
#     if freezing_sim_when_training_adapt:
#       reset_base_network_trainable(False)
#     early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=global_patience, verbose=verbose_param)
#     X_adapt_train, y_adapt_train, X_adapt_valid, y_adapt_valid = split_train_valid_for_pairs(retrieved_X_pairs,y_adapt_diff)
#     early_history = adapt_model.fit(X_adapt_train, y_adapt_train, 
#                         epochs=adapt_per_train_epoch, validation_data=(X_adapt_valid, y_adapt_valid), verbose=verbose_param, 
#                         callbacks=[early_stop, tfdocs.modeling.EpochDots()])
#     #prepare for training of sim model
#     y_adapt_diff_pred = adapt_model.predict([retrieved_X_pairs[0],retrieved_X_pairs[1]])
#     y_pred = retrieved_y + y_adapt_diff_pred
#     #true CC to train sim model
#     y_pair_real = sim_model_label(sim_loss_option, query_y_temp, y_pred)
#     #train sim_model 
#     if freezing_sim_when_training_adapt:
#       reset_base_network_trainable(True)
#     X_sim_train, y_sim_train, X_sim_valid, y_sim_valid = split_train_valid_for_pairs(retrieved_X_pairs,y_pair_real)
#     early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=global_patience, verbose=verbose_param)
#     early_history = sim_model.fit(X_sim_train, y_sim_train, 
#                         epochs=sim_per_train_epoch, validation_data=(X_sim_valid, y_sim_valid), verbose=verbose_param, 
#                         callbacks=[early_stop, tfdocs.modeling.EpochDots()])


##Training Process using all training data until converge!



###Normal mode

In [ ]:
if(batch_mode == False):
  query_indices = np.random.permutation(np.arange(X_train.shape[0]))
  if(num_of_training_queries != -1):
    query_indices = query_indices[:num_of_training_queries]

  # query_counter = 0
  back_forth_counter = 0
  for i in range(back_forth):
    retrieved_X_pairs, retrieved_y, y_adapt_diff, query_y_temp = np.zeros((2,0,x_dimension)),np.zeros((0,1)),np.zeros((0,1)),np.zeros((0,1))
    for j in range(len(query_indices)):
      query_index = query_indices[j]
      #for every query case
      query_X = X_train[query_index]
      query_y = y_train[query_index]
      if(retrieval_debug):
        print("query_X", query_X)
        print("query_y",query_y)

      indices = np.arange(len(X_train))
      #TODO:: to speed up, for efficiency, maybe search/retrieve fewer cases
      retrieve_from_X = X_train[indices!=query_index,:] 
      retrieve_from_y = y_train[indices!=query_index,:] 
      #prepare batch
      temp_retrieved_X_pairs, temp_retrieved_y, temp_y_adapt_diff, temp_query_y_temp = retrieve(query_X, query_y, retrieve_from_X, retrieve_from_y)
      
      retrieved_X_pairs = np.concatenate((retrieved_X_pairs, temp_retrieved_X_pairs),axis=1)#np.vstack((retrieved_X_pairs, temp_retrieved_X_pairs))
      # print(retrieved_y.shape, temp_retrieved_y.shape)
      retrieved_y = np.vstack((retrieved_y, temp_retrieved_y))
      y_adapt_diff = np.vstack((y_adapt_diff, temp_y_adapt_diff))
      query_y_temp = np.vstack((query_y_temp, temp_query_y_temp))

    #actual training when a batch is ready
    if(disable_adaptation == False):
      #train adapt_model
      if freezing_sim_when_training_adapt:
        reset_base_network_trainable(False)
      early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=global_patience, verbose=verbose_param)
      X_adapt_train, y_adapt_train, X_adapt_valid, y_adapt_valid = split_train_valid_for_pairs(retrieved_X_pairs,y_adapt_diff)
      early_history = adapt_model.fit(X_adapt_train, y_adapt_train, 
                          epochs=adapt_per_train_epoch, validation_data=(X_adapt_valid, y_adapt_valid), verbose=verbose_param, 
                          callbacks=[adapt_checkpointer,early_stop, tfdocs.modeling.EpochDots()])
    #prepare for training of sim model
    y_pre_adapt, y_post_adapt = None, None
    if(disable_adaptation):
      y_pre_adapt = sim_model_label(sim_loss_option, query_y_temp, retrieved_y)
      ###BIG BUG
      y_post_adapt = np.copy(y_pre_adapt)
    else:
      y_adapt_diff_pred = adapt_model.predict([retrieved_X_pairs[0],retrieved_X_pairs[1]])
      y_pred = retrieved_y + y_adapt_diff_pred
      #pre-adaptation distance
      y_pre_adapt = sim_model_label(sim_loss_option, query_y_temp, retrieved_y)
      #post-adaptation distance
      y_post_adapt = sim_model_label(sim_loss_option, query_y_temp, y_pred)

    # print("pre ", y_pre_adapt)
    # print("post", y_post_adapt)
    
    y_pair_real = np.concatenate((y_pre_adapt, y_post_adapt), axis = 1)
    # print("y_pair_real", y_pair_real)
    # print("y_pair_real.shape", y_pair_real.shape)

    # y_pair_real = y_pre_adapt

    #train sim_model 
    if(disable_retrieval == False):
      if freezing_sim_when_training_adapt:
        reset_base_network_trainable(True)
      X_sim_train, y_sim_train, X_sim_valid, y_sim_valid = split_train_valid_for_pairs(retrieved_X_pairs,y_pair_real)
      early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=global_patience, verbose=verbose_param)
      early_history = sim_model.fit(X_sim_train, y_sim_train, 
                          epochs=sim_per_train_epoch, validation_data=(X_sim_valid, y_sim_valid), verbose=verbose_param, 
                          callbacks=[sim_checkpointer,early_stop, tfdocs.modeling.EpochDots()])



Epoch 00001: val_loss improved from inf to 0.90179, saving model to adapt_model_weights

Epoch: 0, loss:1.0097,  val_loss:0.9018,  
.
Epoch 00002: val_loss improved from 0.90179 to 0.62197, saving model to adapt_model_weights
.
Epoch 00003: val_loss improved from 0.62197 to 0.47282, saving model to adapt_model_weights
.
Epoch 00004: val_loss improved from 0.47282 to 0.43445, saving model to adapt_model_weights
.
Epoch 00005: val_loss improved from 0.43445 to 0.40792, saving model to adapt_model_weights
.
Epoch 00006: val_loss improved from 0.40792 to 0.38990, saving model to adapt_model_weights
.
Epoch 00007: val_loss improved from 0.38990 to 0.36989, saving model to adapt_model_weights
.
Epoch 00008: val_loss improved from 0.36989 to 0.35136, saving model to adapt_model_weights
.
Epoch 00009: val_loss improved from 0.35136 to 0.34371, saving model to adapt_model_weights
.
Epoch 00010: val_loss did not improve from 0.34371
.
Epoch 00011: val_loss improved from 0.34371 to 0.33814, savi

###Batch mode

In [ ]:
if(batch_mode):
  #prepare validation pairs
  X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size = 0.1, random_state= split_random+1)


  query_indices = np.random.permutation(np.arange(X_train.shape[0]))
  if(num_of_training_queries != -1):
    query_indices = query_indices[:num_of_training_queries]

  query_counter = 0
  retrieved_X_pairs, retrieved_y, y_adapt_diff, query_y_temp = np.zeros((2,0,x_dimension)),np.zeros((0,1)),np.zeros((0,1)),np.zeros((0,1))
  val_X_pairs, val_y, val_y_adapt_diff, val_query_y_temp = np.zeros((2,0,x_dimension)),np.zeros((0,1)),np.zeros((0,1)),np.zeros((0,1))

  back_forth_counter = 0
  for i in range(back_forth):
    for j in range(len(query_indices)):
      query_counter += 1
      query_index = query_indices[j]
      #for every query case
      query_X = X_train[query_index]
      query_y = y_train[query_index]
      if(retrieval_debug):
        print("query_X", query_X)
        print("query_y",query_y)

      indices = np.arange(len(X_train))
      #TODO:: to speed up, for efficiency, maybe search/retrieve fewer cases
      retrieve_from_X = X_train[indices!=query_index,:]
      retrieve_from_y = y_train[indices!=query_index,:]
      #prepare batch
      temp_retrieved_X_pairs, temp_retrieved_y, temp_y_adapt_diff, temp_query_y_temp = retrieve(query_X, query_y, retrieve_from_X, retrieve_from_y)
      retrieved_X_pairs = np.concatenate((retrieved_X_pairs, temp_retrieved_X_pairs),axis=1) #np.vstack((retrieved_X_pairs, temp_retrieved_X_pairs))
      # print(retrieved_y.shape, temp_retrieved_y.shape)
      retrieved_y = np.vstack((retrieved_y, temp_retrieved_y))
      y_adapt_diff = np.vstack((y_adapt_diff, temp_y_adapt_diff))
      query_y_temp = np.vstack((query_y_temp, temp_query_y_temp))

      if(query_counter == batch_size or j == len(query_indices)-1):
        #prepare validation batch
        if((disable_adaptation == False) or (disable_retrieval == False)):
          val_query_indices = np.random.permutation(np.arange(X_val.shape[0]))
          if(num_of_val_queries != -1):
            val_query_indices = val_query_indices[:num_of_val_queries]
          for k in range(len(val_query_indices)):
            val_query_index = val_query_indices[k]
            val_query_X = X_val[val_query_index]
            val_query_y = y_val[val_query_index]
            temp_retrieved_X_pairs, temp_retrieved_y, temp_y_adapt_diff, temp_query_y_temp = retrieve(val_query_X, val_query_y, X_train, y_train)
            val_X_pairs = np.concatenate((val_X_pairs, temp_retrieved_X_pairs),axis=1)
            val_y = np.vstack((val_y, temp_retrieved_y))
            val_y_adapt_diff = np.vstack((val_y_adapt_diff, temp_y_adapt_diff))
            val_query_y_temp = np.vstack((val_query_y_temp, temp_query_y_temp))
          
          X_adapt_valid = val_X_pairs
          y_adapt_valid = val_y_adapt_diff
          # print(y_adapt_valid.shape)
          X_adapt_valid, y_adapt_valid = prepare_pairs(X_adapt_valid, y_adapt_valid)
          # print("X_adapt_valid.shape", X_adapt_valid.shape, y_adapt_valid.shape)

          val_y_pre_adapt, val_y_post_adapt = None, None
          if(disable_adaptation):
            val_y_pre_adapt = sim_model_label(sim_loss_option, val_query_y_temp, val_y)
            val_y_post_adapt = val_y_pre_adapt 
          else:
            y_adapt_diff_pred = adapt_model.predict([val_X_pairs[0],val_X_pairs[1]])
            y_pred = val_y + y_adapt_diff_pred
            #pre-adaptation distance
            val_y_pre_adapt = sim_model_label(sim_loss_option, val_query_y_temp, val_y)
            #post-adaptation distance
            val_y_post_adapt = sim_model_label(sim_loss_option, val_query_y_temp, y_pred)
          # print(val_y_pre_adapt)
          X_sim_valid = val_X_pairs
          y_sim_valid = np.concatenate((val_y_pre_adapt, val_y_post_adapt), axis = 1)
          X_sim_valid, y_sim_valid = prepare_pairs(X_sim_valid, y_sim_valid)
          # print("X_sim_valid.shape", X_sim_valid.shape, y_sim_valid.shape)

        #actual training when a batch is ready
        if(disable_adaptation == False):
          #train adapt_model
          if freezing_sim_when_training_adapt:
            reset_base_network_trainable(False)
          early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=global_patience, verbose=verbose_param)
          # X_adapt_train, y_adapt_train, X_adapt_valid, y_adapt_valid = split_train_valid_for_pairs(retrieved_X_pairs,y_adapt_diff)
          X_adapt_train, y_adapt_train = prepare_pairs(retrieved_X_pairs, y_adapt_diff)
          # print("X_adapt_train.shape", X_adapt_train.shape, y_adapt_train.shape)
          early_history = adapt_model.fit(X_adapt_train, y_adapt_train, batch_size=len(X_adapt_train),#added batch size
                              epochs=adapt_per_train_epoch, validation_data=(X_adapt_valid, y_adapt_valid), verbose=verbose_param, 
                              callbacks=[adapt_checkpointer,early_stop, tfdocs.modeling.EpochDots()])
        #prepare for training of sim model
        y_pre_adapt, y_post_adapt = None, None
        if(disable_adaptation):
          y_pre_adapt = sim_model_label(sim_loss_option, query_y_temp, retrieved_y)
          y_post_adapt = y_pre_adapt #sim_model_label(sim_loss_option, query_y_temp, retrieved_y)
        else:
          y_adapt_diff_pred = adapt_model.predict([retrieved_X_pairs[0],retrieved_X_pairs[1]])
          y_pred = retrieved_y + y_adapt_diff_pred
          #pre-adaptation distance
          y_pre_adapt = sim_model_label(sim_loss_option, query_y_temp, retrieved_y)
          #post-adaptation distance
          y_post_adapt = sim_model_label(sim_loss_option, query_y_temp, y_pred)

        # print("pre ", y_pre_adapt)
        # print("post", y_post_adapt)
        
        y_pair_real = np.concatenate((y_pre_adapt, y_post_adapt), axis = 1)
        # print("y_pair_real", y_pair_real)
        # print("y_pair_real.shape", y_pair_real.shape)

        # y_pair_real = y_pre_adapt

        #train sim_model 
        if(disable_retrieval == False):
          if freezing_sim_when_training_adapt:
            reset_base_network_trainable(True)
          # X_sim_train, y_sim_train, X_sim_valid, y_sim_valid = split_train_valid_for_pairs(retrieved_X_pairs,y_pair_real)
          X_sim_train, y_sim_train = prepare_pairs(retrieved_X_pairs, y_pair_real)
          
          early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=global_patience, verbose=verbose_param)
          early_history = sim_model.fit(X_sim_train, y_sim_train,  batch_size=len(X_sim_train),#added batch size
                              epochs=sim_per_train_epoch, validation_data=(X_sim_valid, y_sim_valid), verbose=verbose_param, 
                              callbacks=[sim_checkpointer,early_stop, tfdocs.modeling.EpochDots()])
        #Reset for next batch
        query_counter = 0
        retrieved_X_pairs, retrieved_y, y_adapt_diff, query_y_temp = np.zeros((2,0,x_dimension)),np.zeros((0,1)),np.zeros((0,1)),np.zeros((0,1))
        val_X_pairs, val_y, val_y_adapt_diff, val_query_y_temp = np.zeros((2,0,x_dimension)),np.zeros((0,1)),np.zeros((0,1)),np.zeros((0,1))


##Checkpoint Load

In [ ]:
#Load the best model
from keras.models import load_model
#because we only load weights, we don't need to worry about loading our special loss function
if(disable_retrieval == False):
  sim_model.load_weights(sim_model_file) #, custom_objects={'mse_loss': mse_loss, 'contrastive_loss':contrastive_loss})
if(disable_adaptation == False):
  adapt_model.load_weights(adapt_model_file)

##Special test

In [ ]:
# disabling_test_on_training_set = True
# if(disabling_test_on_training_set == False):
#   y_preds = np.zeros(y_train.shape)
#   retrieved_y_s = np.zeros(y_train.shape)

#   for query_index in range(X_train.shape[0]):
#     print(query_index)
#     query_X = X_train[query_index]
#     query_y = y_train[query_index]
#     if(retrieval_debug):
#       print("query_X", query_X)
#       print("query_y",query_y)
#     #retrieval
#     retrieved_X_pairs, retrieved_y, y_adapt_diff, query_y = retrieve(query_X, query_y, X_train, y_train)
#     retrieved_y_s[query_index][0] = retrieved_y

#     #adaptation
#     if(disable_adaptation):
#       print(sim_model.predict([retrieved_X_pairs[0], retrieved_X_pairs[0]]))
#       donothing = 0
#     else:
      
#       y_adapt_diff_pred = adapt_model.predict([retrieved_X_pairs[0], retrieved_X_pairs[1]])
#       if(retrieval_debug):
#         print("retrieved_y",retrieved_y)
#         print("y_adapt_diff_pred", y_adapt_diff_pred)
#       y_pred = retrieved_y + y_adapt_diff_pred
#       y_preds[query_index][0] = y_pred[0][0]
#   if(disable_adaptation):
#     y_preds = retrieved_y_s
#   #calculate retrieval loss
#   print("mse_error(y_train, retrieved_y_s)",mse_error(y_train, retrieved_y_s))


#   #calculate real loss
#   print("mse_error(y_train, y_preds)",mse_error(y_train, y_preds))


#Test the models

In [ ]:
#modifying the retrieve size to 1.
#knn will be better than 1nn, but this is not the focus of our research
retrieve_size = 1
bad_retrieve_size = 0

In [ ]:
retrieval_debug = True

TODO, try test on training data set, should get perfect performance??

Test the models on X_test, y_test, assuming the case base contains everything in X_train, y_train

In [ ]:
# X_test = X_train[[529, 3426   ]]
# y_test = y_train[[529,3426    ]]

# X_test = X_test[:100]
# y_test = y_test[:100]

y_preds = np.zeros(y_test.shape)
retrieved_y_s = np.zeros(y_test.shape)

for query_index in range(X_test.shape[0]):
  print(query_index)
  query_X = X_test[query_index]
  query_y = y_test[query_index]
  if(retrieval_debug):
    print("query_X", query_X)
    print("query_y",query_y)
  #retrieval
  retrieved_X_pairs, retrieved_y, y_adapt_diff, query_y = retrieve(query_X, query_y, X_train, y_train)
  retrieved_y_s[query_index][0] = retrieved_y

  #adaptation
  if(disable_adaptation):
    # print(sim_model.predict([retrieved_X_pairs[0], retrieved_X_pairs[0]]))
    donothing = 0
  else:
    
    y_adapt_diff_pred = adapt_model.predict([retrieved_X_pairs[0], retrieved_X_pairs[1]])
    if(retrieval_debug):
      print("retrieved_y",retrieved_y)
      print("y_adapt_diff_pred", y_adapt_diff_pred)
    y_pred = retrieved_y + y_adapt_diff_pred
    y_preds[query_index][0] = y_pred[0][0]
if(disable_adaptation):
  y_preds = retrieved_y_s
#calculate retrieval loss
print("mse_error(y_test, retrieved_y_s)",mse_error(y_test, retrieved_y_s))


#calculate real loss
print("mse_error(y_test, y_preds)",mse_error(y_test, y_preds))


0
query_X [ 1.08316459 -0.63931717  1.79929926  1.31293526 -0.5235573 ]
query_y [-1.26181655]
retrieved_X[0] [-0.28125457 -0.23364856  0.16952016  1.31293526 -0.54221565]
retrieved_y[0] [0.90990842]
retrieved_y [[0.90990842]]
y_adapt_diff_pred [[-1.1586266]]
1
query_X [-0.6620227   0.45936864 -1.18862909  0.29800701 -0.51814647]
query_y [0.47069129]
retrieved_X[0] [ 1.62258612 -0.47028859  0.98440971  0.29800701 -0.52135661]
retrieved_y[0] [-1.785281]
retrieved_y [[-1.785281]]
y_adapt_diff_pred [[0.4373471]]
2
query_X [-0.59856135  0.74671724 -0.91699924  1.31293526  0.22304512]
query_y [1.75528177]
retrieved_X[0] [ 0.353359    0.45936864 -1.18862909  1.31293526 -0.52741631]
retrieved_y[0] [1.36304594]
retrieved_y [[1.36304594]]
y_adapt_diff_pred [[1.3524365]]
3
query_X [-0.71596486  0.2734372  -0.91699924 -0.72334483 -0.41667802]
query_y [0.17531814]
retrieved_X[0] [ 0.08364823 -0.23364856  0.16952016  0.29800701 -0.51780568]
retrieved_y[0] [-0.47110971]
retrieved_y [[-0.47110971]]
y_

In [ ]:
retrieved_y_s

array([[ 0.90990842],
       [-1.785281  ],
       [ 1.36304594],
       [-0.47110971],
       [ 1.21876695],
       [-0.74908239],
       [ 0.29030631],
       [ 0.1087613 ],
       [ 0.30408169],
       [-2.05136335],
       [ 1.256758  ],
       [-0.81302915],
       [ 0.49418194],
       [ 0.50781232],
       [ 0.59495973],
       [-0.37714712],
       [-0.15587101],
       [ 0.75939427],
       [-0.20024223],
       [ 0.56610393],
       [ 0.75359411],
       [ 0.80565054],
       [ 0.52463278],
       [ 0.3551231 ],
       [ 0.62062544],
       [-0.69630093],
       [ 0.14008217],
       [ 0.3551231 ],
       [-0.55898214],
       [ 0.94630443],
       [-1.41479577],
       [-0.93091741],
       [ 0.34569784],
       [-1.44075149],
       [-0.0722037 ],
       [ 0.80304047],
       [ 1.15380516],
       [ 0.3551231 ],
       [ 0.3551231 ],
       [-0.75459254],
       [-0.69630093],
       [-0.63278918],
       [ 0.3551231 ],
       [ 0.3551231 ],
       [ 1.50674491],
       [ 1

In [ ]:
y_test

array([[-1.26181655],
       [ 0.47069129],
       [ 1.75528177],
       [ 0.17531814],
       [ 2.17100825],
       [ 0.6445511 ],
       [ 0.41326971],
       [ 1.003291  ],
       [ 1.03461187],
       [ 1.2386325 ],
       [ 0.39441919],
       [ 0.19039855],
       [ 0.42704509],
       [ 0.11528648],
       [ 0.56363886],
       [-0.62104385],
       [ 0.57842927],
       [ 0.41529977],
       [ 0.66108155],
       [-0.228083  ],
       [ 0.77389467],
       [ 0.42994517],
       [ 0.77534471],
       [-0.23330315],
       [ 0.74547388],
       [ 1.33375513],
       [ 0.60917012],
       [-0.82100438],
       [ 0.39064908],
       [-0.25621378],
       [-1.53384406],
       [ 1.22891723],
       [-2.69358609],
       [ 0.03930438],
       [ 0.95674472],
       [ 0.12935187],
       [ 0.43197523],
       [-0.23475319],
       [ 0.41355972],
       [ 0.84219155],
       [-0.29289979],
       [-0.66295001],
       [ 0.87133736],
       [-0.61321364],
       [-0.25737381],
       [ 1

In [ ]:
print(y_preds)

[[-0.24871813]
 [-1.34793389]
 [ 2.71548248]
 [-0.05303612]
 [ 2.67785373]
 [ 0.07497716]
 [ 0.69485079]
 [ 0.70863294]
 [ 1.3433819 ]
 [-0.63706815]
 [ 1.84904363]
 [-0.33489571]
 [ 0.99032105]
 [ 0.49400739]
 [ 0.84696781]
 [-1.14608834]
 [ 0.45786532]
 [ 1.67405395]
 [ 0.31890022]
 [ 0.08361798]
 [ 1.27523912]
 [ 1.48081793]
 [ 1.34884301]
 [ 0.51157305]
 [ 1.34748703]
 [ 0.37327676]
 [ 0.65244822]
 [ 0.05384675]
 [-0.23373864]
 [ 0.72165095]
 [-3.23323204]
 [ 0.3190715 ]
 [-2.00418404]
 [-1.80100527]
 [ 0.63275766]
 [ 0.75021099]
 [ 2.13294771]
 [ 0.33836562]
 [ 0.54615781]
 [ 0.21542161]
 [-0.93397383]
 [-1.15677903]
 [ 1.08715981]
 [-0.92171484]
 [ 1.29600184]
 [ 2.58663333]
 [-0.66210777]
 [ 0.89527686]
 [ 0.79573905]
 [-3.77036947]
 [ 0.58142321]
 [-1.441651  ]
 [ 0.83293359]
 [-1.94483703]
 [-0.73870205]
 [ 1.08706817]
 [ 2.83526493]
 [ 0.67989879]
 [ 0.83328466]
 [ 2.28466608]
 [-3.02504659]
 [-0.78452915]
 [-0.25393581]
 [ 2.19767556]
 [ 1.71008427]
 [ 0.6234439 ]
 [-0.55243

In [ ]:
print(y_test.shape)
print(y_preds.shape)

(151, 1)
(151, 1)


In [ ]:
#calculate retrieval loss
print("mse_error(y_test, retrieved_y_s)",mse_error(y_test, retrieved_y_s))
#calculate real loss
print("mse_error(y_test, y_preds)",mse_error(y_test, y_preds))

mse_error(y_test, retrieved_y_s) 1.5622630268347912
mse_error(y_test, y_preds) 0.7785380282639087


#Results (Energy Efficiency - Heating)

##In order, Alpha 1

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.04896068801664355
mse_error(y_test, y_preds) 0.03774465913800048

mse_error(y_test, retrieved_y_s) 0.13825905478142503
mse_error(y_test, y_preds) 1.87555500484051

mse_error(y_test, retrieved_y_s) 0.19802488279162295
mse_error(y_test, y_preds) 0.052257921780963536

mse_error(y_test, retrieved_y_s) 0.09375361951280378
mse_error(y_test, y_preds) 0.09776411706868614

mse_error(y_test, retrieved_y_s) 0.0796324572354464
mse_error(y_test, y_preds) 0.4079969373900367

mse_error(y_test, retrieved_y_s) 0.06529916925961432
mse_error(y_test, y_preds) 0.08387142360091963

mse_error(y_test, retrieved_y_s) 0.055432041747754485
mse_error(y_test, y_preds) 0.02945454489732847

mse_error(y_test, retrieved_y_s) 0.02750794405825516
mse_error(y_test, y_preds) 0.11708792019844895

mse_error(y_test, retrieved_y_s) 0.10298399952670723
mse_error(y_test, y_preds) 0.108535432923483

mse_error(y_test, retrieved_y_s) 0.2673427662562399
mse_error(y_test, y_preds) 0.0959604097346806
```

##In order, Beta 1
```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0
beta = 1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.5819128644031599
mse_error(y_test, y_preds) 0.06942077459791646

mse_error(y_test, retrieved_y_s) 0.4941958865461067
mse_error(y_test, y_preds) 0.18520142992357044

mse_error(y_test, retrieved_y_s) 0.9239786450072673
mse_error(y_test, y_preds) 0.03781581223908655

mse_error(y_test, retrieved_y_s) 1.0000517638127195
mse_error(y_test, y_preds) 0.09258420359974057

mse_error(y_test, retrieved_y_s) 2.849121755806663
mse_error(y_test, y_preds) 0.18972715720469874

mse_error(y_test, retrieved_y_s) 2.1501250596762964
mse_error(y_test, y_preds) 0.05748802099312788

mse_error(y_test, retrieved_y_s) 0.30961588764294196
mse_error(y_test, y_preds) 0.3223331002986696

mse_error(y_test, retrieved_y_s) 1.1335240172622465
mse_error(y_test, y_preds) 0.0423217960513657

mse_error(y_test, retrieved_y_s) 1.1382670855503358
mse_error(y_test, y_preds) 0.1428204429882398

mse_error(y_test, retrieved_y_s) 2.3979082220890273
mse_error(y_test, y_preds) 0.3600245907759511
```

##Batch mode, back-n-forth


```
# GLOBAL PARAMETER
adapt_per_train_epoch = 5
sim_per_train_epoch = 5
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0.8
beta = 0.2
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 32
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.03357668300930115
mse_error(y_test, y_preds) 0.028904903126832877

mse_error(y_test, retrieved_y_s) 0.09711864988337508
mse_error(y_test, y_preds) 0.08919824946145367

mse_error(y_test, retrieved_y_s) 0.2089565344086956
mse_error(y_test, y_preds) 0.13920806328534735

mse_error(y_test, retrieved_y_s) 0.15332945656895475
mse_error(y_test, y_preds) 0.11767428288691334

mse_error(y_test, retrieved_y_s) 0.14361975013330525
mse_error(y_test, y_preds) 0.09522780586634487

mse_error(y_test, retrieved_y_s) 0.09119830185025297
mse_error(y_test, y_preds) 0.0732996462391745

mse_error(y_test, retrieved_y_s) 0.1761240411767083
mse_error(y_test, y_preds) 0.13818245536526508

mse_error(y_test, retrieved_y_s) 0.17653006621457035
mse_error(y_test, y_preds) 0.15346412657199138

mse_error(y_test, retrieved_y_s) 0.0795727841102493
mse_error(y_test, y_preds) 0.060813525568383237

mse_error(y_test, retrieved_y_s) 0.20547878671580733
mse_error(y_test, y_preds) 0.16015501122173825
```

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 10
sim_per_train_epoch = 10
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 32
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 1.4862198268696576
mse_error(y_test, y_preds) 0.8041369252130874

mse_error(y_test, retrieved_y_s) 0.3990644334758029
mse_error(y_test, y_preds) 0.2908930312447029

mse_error(y_test, retrieved_y_s) 0.357448342318824
mse_error(y_test, y_preds) 0.13494489658680311
```

#Results (Yacht Hydrodynamics)

##Baseline
```
# GLOBAL PARAMETER
adapt_per_train_epoch = 30
sim_per_train_epoch = 30
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = True
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 0.25116338343839717
mse_error(y_test, y_preds) 0.25116338343839717
```

##Just Retrieval

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.062358762059665354
mse_error(y_test, y_preds) 0.062358762059665354
```

```
mse_error(y_test, retrieved_y_s) 0.0592947882293856
mse_error(y_test, y_preds) 0.0592947882293856
```

With bad retrieval
```
# GLOBAL PARAMETER
adapt_per_train_epoch = 50
sim_per_train_epoch = 50
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 19
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = -1
```

##Just adaptation

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 50
sim_per_train_epoch = 50
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = True
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.168377120060221
mse_error(y_test, y_preds) 0.16837605632714872
```

```
mse_error(y_test, retrieved_y_s) 0.15236735306006033
mse_error(y_test, y_preds) 0.15236782689190728
```

##In order, Alpha 1

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.022469132034920278
mse_error(y_test, y_preds) 0.027133662832116948

mse_error(y_test, retrieved_y_s) 0.04251103345571824
mse_error(y_test, y_preds) 0.062422214023263034

mse_error(y_test, retrieved_y_s) 0.0854176922656829
mse_error(y_test, y_preds) 0.3792443247930353

mse_error(y_test, retrieved_y_s) 0.18851339031485023
mse_error(y_test, y_preds) 0.10574884006328257

mse_error(y_test, retrieved_y_s) 0.04770993670323811
mse_error(y_test, y_preds) 0.27339456880269686

mse_error(y_test, retrieved_y_s) 0.12032400860047178
mse_error(y_test, y_preds) 0.32955364052006897

mse_error(y_test, retrieved_y_s) 0.1620284433875545
mse_error(y_test, y_preds) 0.07804431217921669

mse_error(y_test, retrieved_y_s) 0.06145113438733849
mse_error(y_test, y_preds) 0.07490798881075642

mse_error(y_test, retrieved_y_s) 0.13368304207701964
mse_error(y_test, y_preds) 0.2801522549933505

mse_error(y_test, retrieved_y_s) 0.0983958287974424
mse_error(y_test, y_preds) 0.11268963259325351
```

##In order, Beta 1
```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0
beta = 1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 1.316866920672403
mse_error(y_test, y_preds) 0.23842434234632454

mse_error(y_test, retrieved_y_s) 0.5437332822687408
mse_error(y_test, y_preds) 0.14258684318538195

mse_error(y_test, retrieved_y_s) 1.098907623981999
mse_error(y_test, y_preds) 0.06591577154584774

mse_error(y_test, retrieved_y_s) 1.6094558425249048
mse_error(y_test, y_preds) 0.9838749643069185

mse_error(y_test, retrieved_y_s) 1.8578247003580595
mse_error(y_test, y_preds) 0.11567575916744646

mse_error(y_test, retrieved_y_s) 0.09400500269178277
mse_error(y_test, y_preds) 0.20207439960731496

mse_error(y_test, retrieved_y_s) 2.2346803441918204
mse_error(y_test, y_preds) 0.3129804285880325

mse_error(y_test, retrieved_y_s) 0.8038596350586015
mse_error(y_test, y_preds) 0.09173574761668987

mse_error(y_test, retrieved_y_s) 0.9960577326669695
mse_error(y_test, y_preds) 0.0408670895934164

mse_error(y_test, retrieved_y_s) 1.2770041566182562
mse_error(y_test, y_preds) 0.07285450749732388
```

##Batch mode, back-n-forth


```
# GLOBAL PARAMETER
adapt_per_train_epoch = 10
sim_per_train_epoch = 10
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =0.1
alpha = 0.9
beta = 0.1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 32
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.5533549888856671
mse_error(y_test, y_preds) 0.4729385460332798

mse_error(y_test, retrieved_y_s) 0.04609254331814832
mse_error(y_test, y_preds) 0.06334535612860775

mse_error(y_test, retrieved_y_s) 0.26684819610445465
mse_error(y_test, y_preds) 0.20754980349119398

mse_error(y_test, retrieved_y_s) 0.24263760565984183
mse_error(y_test, y_preds) 0.11553423582083021

mse_error(y_test, retrieved_y_s) 0.11582250821341641
mse_error(y_test, y_preds) 0.05549110194533378

mse_error(y_test, retrieved_y_s) 0.25135820396732733
mse_error(y_test, y_preds) 0.1709596106305274

mse_error(y_test, retrieved_y_s) 0.5781283144010156
mse_error(y_test, y_preds) 0.10875922891995585

mse_error(y_test, retrieved_y_s) 0.1511011463482901
mse_error(y_test, y_preds) 0.12261614939856448

mse_error(y_test, retrieved_y_s) 0.19129156823043675
mse_error(y_test, y_preds) 0.21821602889633052

mse_error(y_test, retrieved_y_s) 0.2403897208882306
mse_error(y_test, y_preds) 0.10684261990040538
```

#Results (Car)

##Baseline: pure 1nn

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 1
sim_per_train_epoch = 3
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.5
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = True
# no adaptation
disable_adaptation = True

batch_mode = False
batch_size = 32
num_of_val_queries = 100
```



```
mse_error(y_test, retrieved_y_s) 0.021406928839911278
mse_error(y_test, y_preds) 0.021406928839911278
```



##Just retrieval 

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = 1000
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 0.22051566612236967
mse_error(y_test, y_preds) 0.22051566612236967
```

##Just adaptation: 1nn+ adaptation

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 40
sim_per_train_epoch = 3
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = 1000
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.5
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = True
# no adaptation
disable_adaptation = False

batch_mode = False
batch_size = 32
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 0.04554783058394877
mse_error(y_test, y_preds) 0.04555953105582095
```

##In order, alpha 1

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = 1000
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 0.13204576052457181
mse_error(y_test, y_preds) 0.10112312536584532
```


```
```

```
mse_error(y_test, retrieved_y_s) 0.670599671011124
mse_error(y_test, y_preds) 0.22546807054123996
```

```
mse_error(y_test, retrieved_y_s) 0.2090307060474565
mse_error(y_test, y_preds) 0.12947463416794605
```

```
mse_error(y_test, retrieved_y_s) 0.24420325113893193
mse_error(y_test, y_preds) 0.36328196360171583
```

```
mse_error(y_test, retrieved_y_s) 0.24420325113893193
mse_error(y_test, y_preds) 0.36328196360171583
```

```
mse_error(y_test, retrieved_y_s) 0.42316665388423197
mse_error(y_test, y_preds) 0.5962574633292452
```

##In order, beta 1

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = 1000
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0
beta = 1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 1.3440959964232262
mse_error(y_test, y_preds) 0.26192260210446044
```

```
mse_error(y_test, retrieved_y_s) 1.1836444471724061
mse_error(y_test, y_preds) 0.33112206936187893
```

```
mse_error(y_test, retrieved_y_s) 3.5374071431685805
mse_error(y_test, y_preds) 0.18891238839110855
```

```
mse_error(y_test, retrieved_y_s) 0.9072356471147421
mse_error(y_test, y_preds) 0.225288185002274
```

```
mse_error(y_test, retrieved_y_s) 1.1069814319422602
mse_error(y_test, y_preds) 0.17702227022632053
```

##Batch mode, no back-n-forth

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 5
sim_per_train_epoch = 5
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = 1000
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.05
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 50
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 1.0201486322341502
mse_error(y_test, y_preds) 0.25953550206147546
```

```
mse_error(y_test, retrieved_y_s) 0.6845301206117389
mse_error(y_test, y_preds) 0.35241454045951737
```

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 5
sim_per_train_epoch = 5
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = 1000
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.05
alpha = 0.9
beta = 0.1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 50
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 0.27950890591069405
mse_error(y_test, y_preds) 0.16286443984315244
```

```
mse_error(y_test, retrieved_y_s) 0.24440905201348648
mse_error(y_test, y_preds) 0.14399497857144386
```

```
mse_error(y_test, retrieved_y_s) 0.8849519385172987
mse_error(y_test, y_preds) 0.5543961065192525
```

```
mse_error(y_test, retrieved_y_s) 0.32507433221595794
mse_error(y_test, y_preds) 0.19996892615797837
```

```
mse_error(y_test, retrieved_y_s) 0.5099855567668763
mse_error(y_test, y_preds) 0.23779425752490574
```

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = 1000
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.05
alpha = 0.9
beta = 0.1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 0.4931109324549002
mse_error(y_test, y_preds) 0.3129729621819068
```

```
mse_error(y_test, retrieved_y_s) 0.21912772664913452
mse_error(y_test, y_preds) 0.1492138345744347
```

#Results (Artificial)

Data setting


```
dimension = 4  #4 #2 #3
value_range = 6  #6 #30 #10
```


##Baseline
```
# GLOBAL PARAMETER
adapt_per_train_epoch = 30
sim_per_train_epoch = 30
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = True
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 0.2727272727272727
mse_error(y_test, y_preds) 0.2727272727272727
```

```
mse_error(y_test, retrieved_y_s) 0.08571428571428572
mse_error(y_test, y_preds) 0.08571428571428572
```

##Just Retrieval

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 9
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 0.006293706293706294
mse_error(y_test, y_preds) 0.006293706293706294
```

##Just adaptation

```

```

##In order, Alpha 1

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = 1000
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 0.05034965034965035
mse_error(y_test, y_preds) 0.18929548431690865

mse_error(y_test, retrieved_y_s) 0.05244755244755246
mse_error(y_test, y_preds) 0.033359415549769164

mse_error(y_test, retrieved_y_s) 0.023076923076923075
mse_error(y_test, y_preds) 0.3383667866439738

mse_error(y_test, retrieved_y_s) 0.0965034965034965
mse_error(y_test, y_preds) 0.0987404551055495

mse_error(y_test, retrieved_y_s) 0.020979020979020976
mse_error(y_test, y_preds) 0.011368292527535866

mse_error(y_test, retrieved_y_s) 0.07342657342657341
mse_error(y_test, y_preds) 0.11548457358265089

mse_error(y_test, retrieved_y_s) 0.027272727272727282
mse_error(y_test, y_preds) 0.011561234601920906

mse_error(y_test, retrieved_y_s) 0.03566433566433566
mse_error(y_test, y_preds) 0.18532339182828628

mse_error(y_test, retrieved_y_s) 0.010489510489510493
mse_error(y_test, y_preds) 0.19836134849201575

mse_error(y_test, retrieved_y_s) 0.006293706293706294
mse_error(y_test, y_preds) 0.056606128212972895
```

##In order, beta 1


```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0
beta = 1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 0.22867132867132872
mse_error(y_test, y_preds) 0.026731099378845324

mse_error(y_test, retrieved_y_s) 0.43006993006993016
mse_error(y_test, y_preds) 0.021653338578586787

mse_error(y_test, retrieved_y_s) 0.3419580419580421
mse_error(y_test, y_preds) 1.0000512101517678

mse_error(y_test, retrieved_y_s) 0.6713286713286715
mse_error(y_test, y_preds) 0.14659527806748449

mse_error(y_test, retrieved_y_s) 1.0930069930069932
mse_error(y_test, y_preds) 0.08789814077126303

mse_error(y_test, retrieved_y_s) 0.7510489510489512
mse_error(y_test, y_preds) 0.5531172436739693

mse_error(y_test, retrieved_y_s) 0.2958041958041959
mse_error(y_test, y_preds) 0.012519394209199192

mse_error(y_test, retrieved_y_s) 0.6566433566433567
mse_error(y_test, y_preds) 0.2411777400610477

mse_error(y_test, retrieved_y_s) 0.5895104895104897
mse_error(y_test, y_preds) 0.01770580725831365

mse_error(y_test, retrieved_y_s) 3.1258741258741267
mse_error(y_test, y_preds) 0.9571460918375673
```

## Batch mode, no back-n-forth




```
# GLOBAL PARAMETER
adapt_per_train_epoch = 2
sim_per_train_epoch = 5
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0.8
beta = 0.2
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 50
num_of_val_queries = -1
```



```
mse_error(y_test, retrieved_y_s) 0.7132867132867132
mse_error(y_test, y_preds) 0.13247966857892654

mse_error(y_test, retrieved_y_s) 0.2937062937062938
mse_error(y_test, y_preds) 0.14991793545825008

mse_error(y_test, retrieved_y_s) 0.15524475524475523
mse_error(y_test, y_preds) 0.118094049414065

mse_error(y_test, retrieved_y_s) 0.22447552447552455
mse_error(y_test, y_preds) 0.21871414960294747

mse_error(y_test, retrieved_y_s) 0.297902097902098
mse_error(y_test, y_preds) 0.1910722710475913
```

Best setting

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 10
sim_per_train_epoch = 10
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 100
num_of_val_queries = -1

```

```
mse_error(y_test, retrieved_y_s) 0.3335664335664337
mse_error(y_test, y_preds) 0.18474562403508557

mse_error(y_test, retrieved_y_s) 0.18881118881118888
mse_error(y_test, y_preds) 0.07264770645732646

mse_error(y_test, retrieved_y_s) 0.26223776223776224
mse_error(y_test, y_preds) 0.10379757019953312

mse_error(y_test, retrieved_y_s) 1.2713286713286716
mse_error(y_test, y_preds) 0.4818192476946495

mse_error(y_test, retrieved_y_s) 0.5034965034965035
mse_error(y_test, y_preds) 0.1964123419478841

mse_error(y_test, retrieved_y_s) 0.297902097902098
mse_error(y_test, y_preds) 0.07500663748709405

mse_error(y_test, retrieved_y_s) 0.19300699300699306
mse_error(y_test, y_preds) 0.07537243572799004

mse_error(y_test, retrieved_y_s) 0.3524475524475525
mse_error(y_test, y_preds) 0.1609173175600261

mse_error(y_test, retrieved_y_s) 0.20559440559440564
mse_error(y_test, y_preds) 0.0985443284065585

mse_error(y_test, retrieved_y_s) 0.28531468531468546
mse_error(y_test, y_preds) 0.135297502286533
```

##Back-n-forth

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 3
sim_per_train_epoch = 3
# how many times the two models are trained for one query
back_forth = 20
 
# retrieval
retrieve_size = 2
bad_retrieve_size = 98
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =0.9
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 1
sim_per_train_epoch = 1
# how many times the two models are trained for one query
back_forth = 20
 
# retrieval
retrieve_size = 2
bad_retrieve_size = 98
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =0.9
alpha = 0.9
beta = 0.1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = -1
```

##Batch mode, back-n-forth

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 3
sim_per_train_epoch = 3
# how many times the two models are trained for one query
back_forth = 30
 
# retrieval
retrieve_size = 2
bad_retrieve_size = 98
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =0.6
alpha = 0.9
beta = 0.1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 32
num_of_val_queries = -1
```

#Results (Student Math ) special, 1 good 4 bad

##Baseline
```
# GLOBAL PARAMETER
adapt_per_train_epoch = 30
sim_per_train_epoch = 30
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = True
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 1.4126062568151605
mse_error(y_test, y_preds) 1.4126062568151605
```

```
mse_error(y_test, retrieved_y_s) 1.5367914222494607
mse_error(y_test, y_preds) 1.5367914222494607
```

```
mse_error(y_test, retrieved_y_s) 2.9087986826726384
mse_error(y_test, y_preds) 2.9087986826726384
```

##Just Retrieval

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 4
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.9
alpha = 1.0
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 20
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 1.3421550571937795
mse_error(y_test, y_preds) 1.3421550571937795
```

```
mse_error(y_test, retrieved_y_s) 1.356484114743891
mse_error(y_test, y_preds) 1.356484114743891
```

```
mse_error(y_test, retrieved_y_s) 1.0961729025835316
mse_error(y_test, y_preds) 1.0961729025835316
```

```
mse_error(y_test, retrieved_y_s) 0.8000390465478933
mse_error(y_test, y_preds) 0.8000390465478933
```

##Just adaptation


```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 4
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.9
alpha = 1.0
beta = 0
 
# use naive 1nn instead
disable_retrieval = True
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 20
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 1.6526179707795294
mse_error(y_test, y_preds) 1.0542575371845913
```


```
mse_error(y_test, retrieved_y_s) 1.6526179707795294
mse_error(y_test, y_preds) 0.8916261505858021
```

##In order, Alpha 1

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 4
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.9
alpha = 1.0
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 20
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 1.3218555589977878
mse_error(y_test, y_preds) 1.174342375289472
```

```
mse_error(y_test, retrieved_y_s) 1.1725945428507931
mse_error(y_test, y_preds) 1.0031874768894329
```

##In order, Beta 1


```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 4
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.9
alpha = 0
beta = 1.0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 20
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 1.651423882650353
mse_error(y_test, y_preds) 1.1059572724470328
```

```
mse_error(y_test, retrieved_y_s) 1.2848388269933335
mse_error(y_test, y_preds) 0.8633932318513875
```

```
mse_error(y_test, retrieved_y_s) 1.2072230985968957
mse_error(y_test, y_preds) 0.9356769442244126
```

## Batch mode, no back-n-forth




```
# GLOBAL PARAMETER
adapt_per_train_epoch = 1
sim_per_train_epoch = 2
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 4
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =1.2
alpha = 0.5
beta = 1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 8
num_of_val_queries = -1
```



```
mse_error(y_test, retrieved_y_s) 1.4627579582405514
mse_error(y_test, y_preds) 1.385761350677766
```

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 3
sim_per_train_epoch = 3
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 4
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =1.2
alpha = 0.5
beta = 1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 16
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 1.5045510427617101
mse_error(y_test, y_preds) 1.254594655735692
```

```
mse_error(y_test, retrieved_y_s) 1.234687125567943
mse_error(y_test, y_preds) 1.032116667311885
```

```
mse_error(y_test, retrieved_y_s) 2.16368769006684
mse_error(y_test, y_preds) 1.4308408161541615
```

#Results (Student Math )

##Baseline
```
# GLOBAL PARAMETER
adapt_per_train_epoch = 30
sim_per_train_epoch = 30
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = True
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 1.4126062568151605
mse_error(y_test, y_preds) 1.4126062568151605
```

```
mse_error(y_test, retrieved_y_s) 1.5367914222494607
mse_error(y_test, y_preds) 1.5367914222494607
```

```
mse_error(y_test, retrieved_y_s) 2.9087986826726384
mse_error(y_test, y_preds) 2.9087986826726384
```

##Just Retrieval

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 30
sim_per_train_epoch = 30
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 2.0072621451447894
mse_error(y_test, y_preds) 2.0072621451447894
```

```
mse_error(y_test, retrieved_y_s) 1.4102180805568092
mse_error(y_test, y_preds) 1.4102180805568092
```

With bad retrieval
```
# GLOBAL PARAMETER
adapt_per_train_epoch = 30
sim_per_train_epoch = 30
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 2
bad_retrieve_size = 18
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 0.9254183001113692
mse_error(y_test, y_preds) 0.9254183001113692
```

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 2
bad_retrieve_size = 98
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 1.227522596792887
mse_error(y_test, y_preds) 1.227522596792887
```

##Just adaptation

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 30
sim_per_train_epoch = 30
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = True
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```



```
mse_error(y_test, retrieved_y_s) 1.6526179707795294
mse_error(y_test, y_preds) 1.6526172269003685
```



```
mse_error(y_test, retrieved_y_s) 1.6526179707795294
mse_error(y_test, y_preds) 1.6526247169717894
```

##In order, Alpha 1

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 1.6729174689755204
mse_error(y_test, y_preds) 1.6321990249865443

mse_error(y_test, retrieved_y_s) 1.1391600752338662
mse_error(y_test, y_preds) 1.1166571940090972

mse_error(y_test, retrieved_y_s) 2.3296659400222985
mse_error(y_test, y_preds) 2.2415303987209048

mse_error(y_test, retrieved_y_s) 1.3337964402895472
mse_error(y_test, y_preds) 1.271038320280446

mse_error(y_test, retrieved_y_s) 1.4961924258574784
mse_error(y_test, y_preds) 1.206510744438814

mse_error(y_test, retrieved_y_s) 1.5582850085746283
mse_error(y_test, y_preds) 1.6165487919304884

mse_error(y_test, retrieved_y_s) 2.071742904120291
mse_error(y_test, y_preds) 1.4185128543100194

mse_error(y_test, retrieved_y_s) 1.7433686685969019
mse_error(y_test, y_preds) 1.5836352162207592

mse_error(y_test, retrieved_y_s) 1.4388761956570322
mse_error(y_test, y_preds) 1.4396716392448041

mse_error(y_test, retrieved_y_s) 1.6203775912917782
mse_error(y_test, y_preds) 2.283089111191994
```

```
mse_error(y_test, retrieved_y_s) 1.6669470283296406
mse_error(y_test, y_preds) 1.6006473927642162
```

```
mse_error(y_test, retrieved_y_s) 1.5523145679287484
mse_error(y_test, y_preds) 4.997893834840976
```

```
mse_error(y_test, retrieved_y_s) 2.6138589147661775
mse_error(y_test, y_preds) 2.067906309082334
```

```
mse_error(y_test, retrieved_y_s) 1.2860329151225094
mse_error(y_test, y_preds) 1.3977059718973683
```

```
mse_error(y_test, retrieved_y_s) 1.8914355966147212
mse_error(y_test, y_preds) 1.3904559586718777
```

##In order, Beta 1

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0
beta = 1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 1.4675343107572554
mse_error(y_test, y_preds) 1.2006594612156296

mse_error(y_test, retrieved_y_s) 1.7672504311804214
mse_error(y_test, y_preds) 0.9691151046373017

mse_error(y_test, retrieved_y_s) 1.7565036380178376
mse_error(y_test, y_preds) 1.6699511013292807

mse_error(y_test, retrieved_y_s) 2.2711556216926767
mse_error(y_test, y_preds) 2.178678870346171

mse_error(y_test, retrieved_y_s) 1.4233530499777447
mse_error(y_test, y_preds) 3.2462252947794092

mse_error(y_test, retrieved_y_s) 1.7027696722049193
mse_error(y_test, y_preds) 1.9435881493442302

mse_error(y_test, retrieved_y_s) 2.0574138465701792
mse_error(y_test, y_preds) 1.262679355921289

mse_error(y_test, retrieved_y_s) 2.16965813071272
mse_error(y_test, y_preds) 1.6970027868394086

mse_error(y_test, retrieved_y_s) 1.2108053629844238
mse_error(y_test, y_preds) 1.1744281004500028

mse_error(y_test, retrieved_y_s) 2.1565231612917843
mse_error(y_test, y_preds) 2.0666987457666957
```

```
mse_error(y_test, retrieved_y_s) 6.354937023474461
mse_error(y_test, y_preds) 4.344867924628907
```

```
mse_error(y_test, retrieved_y_s) 1.7159046416258548
mse_error(y_test, y_preds) 3.6871850663185426
```

```
mse_error(y_test, retrieved_y_s) 1.7373982279510223
mse_error(y_test, y_preds) 1.1121339185086203
```

```
mse_error(y_test, retrieved_y_s) 1.440070283786208
mse_error(y_test, y_preds) 1.208382987582968
```

```
mse_error(y_test, retrieved_y_s) 1.5761963305122677
mse_error(y_test, y_preds) 1.6583771542264116
```

```
mse_error(y_test, retrieved_y_s) 1.8496425120935627
mse_error(y_test, y_preds) 1.5419585427708093
```

## Batch mode, no back-n-forth



Best setting

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 10
sim_per_train_epoch = 10
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =1.2
alpha = 0.8
beta = 0.2
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 32
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 1.2824506507349813
mse_error(y_test, y_preds) 2.355443466644986

mse_error(y_test, retrieved_y_s) 1.352901850356363
mse_error(y_test, y_preds) 1.5259045170610241

mse_error(y_test, retrieved_y_s) 1.5188801003118215
mse_error(y_test, y_preds) 1.1126197166018363

mse_error(y_test, retrieved_y_s) 1.474698839532311
mse_error(y_test, y_preds) 1.8344448316695818

mse_error(y_test, retrieved_y_s) 1.1296073702004583
mse_error(y_test, y_preds) 1.175477580869368

mse_error(y_test, retrieved_y_s) 1.6729174689755204
mse_error(y_test, y_preds) 1.8071185929617044

mse_error(y_test, retrieved_y_s) 0.8740725105568028
mse_error(y_test, y_preds) 0.8156386359570348

mse_error(y_test, retrieved_y_s) 1.9439754742984632
mse_error(y_test, y_preds) 1.64272810018386

mse_error(y_test, retrieved_y_s) 2.1756285713585997
mse_error(y_test, y_preds) 2.2301503832435285

mse_error(y_test, retrieved_y_s) 1.435293931269504
mse_error(y_test, y_preds) 1.5661549748030925
```

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 5
sim_per_train_epoch = 5
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =1.2
alpha = 0.8
beta = 0.2
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 16
num_of_val_queries = -1

```

```
mse_error(y_test, retrieved_y_s) 2.0120384976614933
mse_error(y_test, y_preds) 1.9627106924870836

mse_error(y_test, retrieved_y_s) 1.5033569546325338
mse_error(y_test, y_preds) 1.3513416940929992

mse_error(y_test, retrieved_y_s) 1.4890278970824224
mse_error(y_test, y_preds) 1.1975711922377212

mse_error(y_test, retrieved_y_s) 2.5625131252116105
mse_error(y_test, y_preds) 1.5712899230370234

mse_error(y_test, retrieved_y_s) 2.0657724634744112
mse_error(y_test, y_preds) 2.1208465262574796
```

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 3
sim_per_train_epoch = 3
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =1.2
alpha = 0.8
beta = 0.2
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 16
num_of_val_queries = -1
```



```
mse_error(y_test, retrieved_y_s) 1.7887440175055886
mse_error(y_test, y_preds) 1.8516919733755415

mse_error(y_test, retrieved_y_s) 1.7947144581514685
mse_error(y_test, y_preds) 1.8606819036444773

mse_error(y_test, retrieved_y_s) 1.3576782028730665
mse_error(y_test, y_preds) 1.1857100718647176

mse_error(y_test, retrieved_y_s) 1.3564841147438909
mse_error(y_test, y_preds) 1.330388545422439

mse_error(y_test, retrieved_y_s) 1.777997224343005
mse_error(y_test, y_preds) 2.026588278782596
```

```
mse_error(y_test, retrieved_y_s) 1.2024467460801922
mse_error(y_test, y_preds) 1.1019735986730446

mse_error(y_test, retrieved_y_s) 1.4591756938530236
mse_error(y_test, y_preds) 1.465053875559009

mse_error(y_test, retrieved_y_s) 1.5582850085746283
mse_error(y_test, y_preds) 1.538326632230536

mse_error(y_test, retrieved_y_s) 1.7982967225389963
mse_error(y_test, y_preds) 1.794444109501626

mse_error(y_test, retrieved_y_s) 2.500420542494461
mse_error(y_test, y_preds) 2.5070646017186284
```

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 3
sim_per_train_epoch = 3
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.9
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 100
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 2.2401093303341013
mse_error(y_test, y_preds) 2.1895050026409812
```

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 1
sim_per_train_epoch = 1
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =0.9
alpha = 0.9
beta = 0.1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 32
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 2.144582280000025
mse_error(y_test, y_preds) 2.1402503950788985
```

#Results (Student Math ) Special, with 1 bad retrieval



##Baseline
```
# GLOBAL PARAMETER
adapt_per_train_epoch = 30
sim_per_train_epoch = 30
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = True
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 1.4126062568151605
mse_error(y_test, y_preds) 1.4126062568151605
```

```
mse_error(y_test, retrieved_y_s) 1.5367914222494607
mse_error(y_test, y_preds) 1.5367914222494607
```

```
mse_error(y_test, retrieved_y_s) 2.9087986826726384
mse_error(y_test, y_preds) 2.9087986826726384
```

##Just Retrieval

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 1
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 1.5845549474164993
mse_error(y_test, y_preds) 1.5845549474164993
```

##Just adaptation

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 1
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = True
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 1.6526179707795294
mse_error(y_test, y_preds) 1.2500101713012022
```

##In order, Alpha 1

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 1
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 1.6430652657461213
mse_error(y_test, y_preds) 1.0184390118000455
```

##In order, Beta 1

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 1
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0
beta = 1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 1.9857685588196219
mse_error(y_test, y_preds) 1.01699471833626
```

```
mse_error(y_test, retrieved_y_s) 1.633512560712714
mse_error(y_test, y_preds) 0.8960150015602497
```

## Batch mode, no back-n-forth


```
# GLOBAL PARAMETER
adapt_per_train_epoch = 3
sim_per_train_epoch = 3
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 1
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =0.9
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 16
num_of_val_queries = -1
```


```
mse_error(y_test, retrieved_y_s) 1.5451500391536928
mse_error(y_test, y_preds) 1.41503107761728
```

```
mse_error(y_test, retrieved_y_s) 2.0526374940534757
mse_error(y_test, y_preds) 1.6673338121437176
```

```
mse_error(y_test, retrieved_y_s) 1.3075265014476767
mse_error(y_test, y_preds) 1.099742346619724
```

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 50
sim_per_train_epoch = 50
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 1
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.9
alpha = 0.8
beta = 0.2
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 50
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 1.4567875175946718
mse_error(y_test, y_preds) 1.3732194050310178
```

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 2
sim_per_train_epoch = 2
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 1
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.9
alpha = 0.8
beta = 0.2
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 32
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 1.6120189743875464
mse_error(y_test, y_preds) 1.347297598825745
```

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 3
sim_per_train_epoch = 3
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 1
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.9
alpha = 0.8
beta = 0.2
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 32
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 2.260408828530093
mse_error(y_test, y_preds) 2.1299467068023303
```

#Results (Airfoil )

##Baseline
```
# GLOBAL PARAMETER
adapt_per_train_epoch = 30
sim_per_train_epoch = 30
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = True
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 0.15010504121565116
mse_error(y_test, y_preds) 0.15010504121565116
```

```
mse_error(y_test, retrieved_y_s) 0.13988533026432518
mse_error(y_test, y_preds) 0.13988533026432518
```

##Just Retrieval

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.2982626040079169
mse_error(y_test, y_preds) 0.2982626040079169
```

With bad retrieval
```
# GLOBAL PARAMETER
adapt_per_train_epoch = 50
sim_per_train_epoch = 50
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 19
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.04494547605601288
mse_error(y_test, y_preds) 0.04494547605601288
```

##Just adaptation

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 50
sim_per_train_epoch = 50
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = True
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.13988533026432523
mse_error(y_test, y_preds) 0.13988530189240211
```

##In order, Alpha 1

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.4425462464188712
mse_error(y_test, y_preds) 0.4024145520164575

mse_error(y_test, retrieved_y_s) 0.6905800924018325
mse_error(y_test, y_preds) 1.547051959717854

mse_error(y_test, retrieved_y_s) 1.3609996295357503
mse_error(y_test, y_preds) 0.6377161575807369

mse_error(y_test, retrieved_y_s) 0.7790365306245126
mse_error(y_test, y_preds) 0.6391381821293315

mse_error(y_test, retrieved_y_s) 0.598782509431653
mse_error(y_test, y_preds) 0.4767601292214646

mse_error(y_test, retrieved_y_s) 0.1664561416164127
mse_error(y_test, y_preds) 0.15577071698394268

mse_error(y_test, retrieved_y_s) 0.2041661866873775
mse_error(y_test, y_preds) 0.5681454623505945

mse_error(y_test, retrieved_y_s) 1.1664733555930598
mse_error(y_test, y_preds) 1.8098747731845275

mse_error(y_test, retrieved_y_s) 0.45291641946612615
mse_error(y_test, y_preds) 0.30283930299666356

mse_error(y_test, retrieved_y_s) 0.4434972616416057
mse_error(y_test, y_preds) 0.39761149140542335
```

```
mse_error(y_test, retrieved_y_s) 0.25641431840408846
mse_error(y_test, y_preds) 2.7285021907993263
```

```
mse_error(y_test, retrieved_y_s) 0.24183877929154426
mse_error(y_test, y_preds) 0.21239972079443736
```

```
mse_error(y_test, retrieved_y_s) 0.23076767493023514
mse_error(y_test, y_preds) 0.1558808527192187
```

```
mse_error(y_test, retrieved_y_s) 0.35459495183372314
mse_error(y_test, y_preds) 0.29055949953798077
```

```
mse_error(y_test, retrieved_y_s) 0.40383703941857213
mse_error(y_test, y_preds) 0.4821457518197068
```

##In order, Beta 1
```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 0
beta = 1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.46403524973231774
mse_error(y_test, y_preds) 0.20509478968437406

mse_error(y_test, retrieved_y_s) 0.45903314560489705
mse_error(y_test, y_preds) 0.14048577465136255

mse_error(y_test, retrieved_y_s) 0.3628531817737483
mse_error(y_test, y_preds) 0.16474354691042398

mse_error(y_test, retrieved_y_s) 1.9562225883814601
mse_error(y_test, y_preds) 0.1486296982282566

mse_error(y_test, retrieved_y_s) 1.4082829827386398
mse_error(y_test, y_preds) 0.8829018096664359

mse_error(y_test, retrieved_y_s) 2.157250886880225
mse_error(y_test, y_preds) 0.20728651604318316

mse_error(y_test, retrieved_y_s) 1.3287368249579947
mse_error(y_test, y_preds) 0.26743847250510044

mse_error(y_test, retrieved_y_s) 1.0547595304874833
mse_error(y_test, y_preds) 0.26288981239558673

mse_error(y_test, retrieved_y_s) 0.8380843726425858
mse_error(y_test, y_preds) 0.17617313783025046

mse_error(y_test, retrieved_y_s) 1.5622630268347912
mse_error(y_test, y_preds) 0.7785380282639087

mse_error(y_test, retrieved_y_s) 4.658178845135962
mse_error(y_test, y_preds) 1.2367158857139586
```

```
mse_error(y_test, retrieved_y_s) 0.35970780005621167
mse_error(y_test, y_preds) 0.1516203730380604
```

```
mse_error(y_test, retrieved_y_s) 1.1476596831619883
mse_error(y_test, y_preds) 0.4573007348698502
```

```
mse_error(y_test, retrieved_y_s) 1.17240420406107
mse_error(y_test, y_preds) 0.2961135748623461
```

```
mse_error(y_test, retrieved_y_s) 2.623373018345764
mse_error(y_test, y_preds) 0.6778236477257922
```

```
mse_error(y_test, retrieved_y_s) 1.4400979981731359
mse_error(y_test, y_preds) 0.158297125077583
```

##Batch mode, back-n-forth

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 20
sim_per_train_epoch = 20
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =0.15
alpha = 0.8
beta = 0.2
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 32
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.39328078526200294
mse_error(y_test, y_preds) 0.29408781711633175
```

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 1
sim_per_train_epoch = 1
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =0.15
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 32
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.5000880033049067
mse_error(y_test, y_preds) 0.38623735292095096
```

```
mse_error(y_test, retrieved_y_s) 0.3370969156929242
mse_error(y_test, y_preds) 0.3382548697224044
```

```
mse_error(y_test, retrieved_y_s) 0.20253714627344643
mse_error(y_test, y_preds) 0.2021226488830182
```

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 3
sim_per_train_epoch = 3
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.9
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 32
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.35301470172187044
mse_error(y_test, y_preds) 0.24633832246173515
```

```
mse_error(y_test, retrieved_y_s) 0.26671536481319985
mse_error(y_test, y_preds) 0.25422066595255954
```

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 20
sim_per_train_epoch = 20
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =0.15
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 32
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.9913191826067004
mse_error(y_test, y_preds) 0.9052194450656151
```

Best setting

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 3
sim_per_train_epoch = 3
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'easy'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =0.2
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 32
num_of_val_queries = -1
```

```
mse_error(y_test, retrieved_y_s) 0.4369289138891386
mse_error(y_test, y_preds) 0.1994561611746476

mse_error(y_test, retrieved_y_s) 0.24605958426828073
mse_error(y_test, y_preds) 0.24722827783979717

mse_error(y_test, retrieved_y_s) 0.21780320554756696
mse_error(y_test, y_preds) 0.16483680580831062

mse_error(y_test, retrieved_y_s) 0.18118761826006846
mse_error(y_test, y_preds) 0.1670973088062938

mse_error(y_test, retrieved_y_s) 0.277211254531777
mse_error(y_test, y_preds) 0.21403847882035568

mse_error(y_test, retrieved_y_s) 0.3210448013175407
mse_error(y_test, y_preds) 0.24014046546044449

mse_error(y_test, retrieved_y_s) 0.39438900515536807
mse_error(y_test, y_preds) 0.21582265125280972

mse_error(y_test, retrieved_y_s) 0.2881489774201352
mse_error(y_test, y_preds) 0.24313228917019766

mse_error(y_test, retrieved_y_s) 0.1976094075009802
mse_error(y_test, y_preds) 0.20360597385104856

mse_error(y_test, retrieved_y_s) 0.21006593245004052
mse_error(y_test, y_preds) 0.1956599561434689
```

```
mse_error(y_test, retrieved_y_s) 0.4291156445863231
mse_error(y_test, y_preds) 0.22480368477621052
```

```
mse_error(y_test, retrieved_y_s) 0.22366279691208846
mse_error(y_test, y_preds) 0.19860016385119122
```

```
mse_error(y_test, retrieved_y_s) 0.23294931460989898
mse_error(y_test, y_preds) 0.14107563349022342
```

```
mse_error(y_test, retrieved_y_s) 0.26536031407856614
mse_error(y_test, y_preds) 0.1685571298052191
```

```
mse_error(y_test, retrieved_y_s) 0.28093424968462755
mse_error(y_test, y_preds) 0.19585244600040438
```

#Results (Face)

##Baseline: pure 1nn

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 1
sim_per_train_epoch = 3
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = -1
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.5
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = True
# no adaptation
disable_adaptation = True

batch_mode = False
batch_size = 32
num_of_val_queries = 100
```

```
mse_error(y_test, retrieved_y_s) 1.0855163609032057
mse_error(y_test, y_preds) 1.0855163609032057
```

For following experiment, I have to remove part of the dataset to prevent crashing

##Just retrieval 

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 2
bad_retrieve_size = 8
 
num_of_training_queries = 1000
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = True
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

##Just adaptation: 1nn+ adaptation

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 40
sim_per_train_epoch = 3
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = 1000
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.5
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = True
# no adaptation
disable_adaptation = False

batch_mode = False
batch_size = 32
num_of_val_queries = 100
```

setting
```
# GLOBAL PARAMETER
adapt_per_train_epoch = 30
sim_per_train_epoch = 30
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 1
bad_retrieve_size = 0
 
num_of_training_queries = 1000
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.5
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = True
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 32
num_of_val_queries = 100
```

##In order, alpha 1

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 2
bad_retrieve_size = 8
 
num_of_training_queries = 1000
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0.1
alpha = 1
beta = 0
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

##In order, alpha 0.5, beta 0.5

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 2
bad_retrieve_size = 8
 
num_of_training_queries = 1000
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = False
alpha_margin =0
alpha = 0.5
beta = 0.5
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

##Back-n-forth

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 3
sim_per_train_epoch = 3
# how many times the two models are trained for one query
back_forth = 20
 
# retrieval
retrieve_size = 2
bad_retrieve_size = 8
 
num_of_training_queries = 1000
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =0.05
alpha = 0.9
beta = 0.1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = False
batch_size = 100
num_of_val_queries = 100
```

##Batch mode, no back-n-forth

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 100
sim_per_train_epoch = 100
# how many times the two models are trained for one query
back_forth = 1
 
# retrieval
retrieve_size = 2
bad_retrieve_size = 8
 
num_of_training_queries = 1000
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =0.03
alpha = 0.9
beta = 0.1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 100
num_of_val_queries = 100
```

##Batch mode, back-n-forth

```
# GLOBAL PARAMETER
adapt_per_train_epoch = 3
sim_per_train_epoch = 3
# how many times the two models are trained for one query
back_forth = 20
 
# retrieval
retrieve_size = 2
bad_retrieve_size = 8
 
num_of_training_queries = 1000
# model choice
hard_or_easy = 'hard'#'hard' 'easy'
model_separation = True
 
using_margin = True
alpha_margin =0.05
alpha = 0.9
beta = 0.1
 
# use naive 1nn instead
disable_retrieval = False
# no adaptation
disable_adaptation = False
 
batch_mode = True
batch_size = 100
num_of_val_queries = 100
```